In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple
import numpy as np
# from geneticalgorithm import geneticalgorithm as ga
import pandas as pd
from collections import Counter
from tqdm import tqdm
import time
from Bio.SeqUtils import MeltingTemp
from Bio import SeqIO
from plotly import graph_objects as go
import json
from imp import reload
# import primer_selection
# reload(primer_selection)
# import testing
# reload(testing)
import math
from itertools import combinations

from primer3 import calc_heterodimer


/tmp/ipykernel_2480574/2667737495.py:18: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [44]:
def extract_gff3(gff3_file_path):
    # Read the GFF3 file
    gff3_columns = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    gff3_df = pd.read_csv(gff3_file_path, sep='\t', comment='#', names=gff3_columns)

    # Filter to keep only gene entries and explicitly create a copy
    genes_df = gff3_df[gff3_df['type'] == 'gene'].copy()

    # Extract gene names from the attributes column
    genes_df['gene_id'] = genes_df['attributes'].apply(lambda x: x.split(';')[0].split('=')[1].split(':')[1])
    genes_df['gene_name'] = genes_df['attributes'].apply(lambda x: x.split(';')[1].split('=')[1])

    # Create a new DataFrame with the required information
    genes_info_df = genes_df[['gene_id', 'gene_name', 'seqid', 'start', 'end']]

    # Rename columns for clarity
    genes_info_df.columns = ['gene_id', 'gene_name', 'chr', 'start', 'end']
    # genes_info_df = genes_info_df[genes_info_df['gene_name'] != 'protein_coding']
    
    return genes_info_df

GF = extract_gff3('/mnt/storage10/lwang/Projects/TOAST/toast/db/MTB-h37rv_asm19595v2-eg18.gff')

In [19]:
accepted_primers = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output_seg1/Amplicon_design_output/Primer_design-accepted_primers-33-300_800_50_3.csv')
accepted_primers = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output_seg/Amplicon_design_output/Primer_design-accepted_primers-33-300_800_50_3.csv')


In [8]:
1471903

for x, row in GF.iterrows():
    if row['start'] >= 1471903 and 1471903 <= row['end']:
        print(row)
        break

gene_id         Rv1316c
gene_name           ogt
chr          Chromosome
start           1477134
end             1477631
Name: 5540, dtype: object


In [48]:
GF

gene_id       gene_name         chr    start      end
0       Rv0001            dnaA  Chromosome        1     1524
5       Rv0002            dnaN  Chromosome     2052     3260
9       Rv0003            recF  Chromosome     3280     4437
13      Rv0004  protein_coding  Chromosome     4434     4997
17      Rv0005            gyrB  Chromosome     5240     7267
...        ...             ...         ...      ...      ...
16465  Rv3920c  protein_coding  Chromosome  4408334  4408897
16469  Rv3921c  protein_coding  Chromosome  4408969  4410069
16473  Rv3922c  protein_coding  Chromosome  4410053  4410415
16477  Rv3923c            rnpA  Chromosome  4410412  4410789
16481  Rv3924c            rpmH  Chromosome  4410786  4410929

[4018 rows x 5 columns]

In [53]:
gene_ = []

for i, row in accepted_primers.iterrows():
    _ = False
    _ = GF[GF['start'] <= row['pRight_coord']]
    _ = _[_['end'] >= row['pLeft_coord']].iloc[:1]
    if _.shape[0]>0:
        gene_.append(_['gene_name'].values[0])
        print(_.shape)
        print(row['Amplicon_ID'], row['pLeft_coord'], row['pRight_coord'],  _['gene_id'].values, _['gene_name'].values)
    else:
        gene_.append('_')
    print('--------------------------')
# accepted_primers['Amplicon_ID'] = [str(g) + '-' + str(a) for g, a in zip(gene_, accepted_primers['Amplicon_ID'])]
# new_amplicon_id = [str(g) + '-' + str(a) for g, a in zip(gene_, accepted_primers['Amplicon_ID'])]


(1, 5)
protein_coding-A1-sp 2153871 2154429 ['Rv1907c'] ['protein_coding']
--------------------------
(1, 5)
katG-A2-sp 2154365 2155295 ['Rv1908c'] ['katG']
--------------------------
(1, 5)
katG-A3-sp 2155277 2156154 ['Rv1908c'] ['katG']
--------------------------
(1, 5)
mmpS5-A4-sp 778851 779604 ['Rv0677c'] ['mmpS5']
--------------------------
(1, 5)
rpoB-A1-ns 760602 761526 ['Rv0667'] ['rpoB']
--------------------------
(1, 5)
protein_coding-A2-ns 2288500 2289343 ['Rv2042c'] ['protein_coding']
--------------------------
(1, 5)
embB-A3-ns 4247277 4248109 ['Rv3795'] ['embB']
--------------------------
(1, 5)
protein_coding-A4-ns 781291 782142 ['Rv0681'] ['protein_coding']
--------------------------
(1, 5)
fabG1-A5-ns 1673325 1674124 ['Rv1483'] ['fabG1']
--------------------------
(1, 5)
gyrB-A6-ns 7026 7800 ['Rv0005'] ['gyrB']
--------------------------
--------------------------
(1, 5)
parA-A8-ns 4407478 4408236 ['Rv3918c'] ['parA']
--------------------------
(1, 5)
ethA-A9-ns 432616

In [31]:
for x, row in GF.iterrows():
    if (1472598 >= row['start']) and (1471903 <= row['end']):
        print(row)
        # break

In [33]:
for x, row in GF.iterrows():
    if row['start'] == 1471903 and row['end'] == 1471903:
        print(row)
        # break

In [35]:
for x, row in GF.iterrows():
    if row['start'] >= 1471903 and 1471903 <= row['end']:
        print(row)
        break

gene_id         Rv1316c
gene_name           ogt
chr          Chromosome
start           1477134
end             1477631
Name: 5540, dtype: object


In [41]:
    _ = GF[GF['start'] <= 1472598]
    _ = _[_['end'] >= 1471903]

In [42]:
_

Empty DataFrame
Columns: [gene_id, gene_name, chr, start, end]
Index: []

In [17]:
    _ = GF[GF['start'] <= 1475826]
    _ = _[_['end'] >= 1476629]

In [18]:
_

Empty DataFrame
Columns: [gene_id, gene_name, chr, start, end]
Index: []

In [14]:
gene_

['katG',
 'katG',
 'katG',
 'mmpS5',
 'rpoB',
 'pncA',
 'embB',
 'rpsL',
 'fabG1',
 'fadA5',
 '_',
 'gyrB',
 '_',
 'gid',
 'rplC',
 '_',
 'eis',
 'ethA',
 'ethA',
 'embC',
 'inhA',
 'ethA',
 'ahpC',
 'folC',
 'gyrB',
 'embB',
 'rraA',
 'alr',
 'thyX',
 'tlyA',
 'thyA',
 'rpoB',
 'thyA']

In [62]:
accepted_primers

Amplicon_ID   pLeft_ID  pLeft_coord  pLeft_length  pLeft_Tm  \
0      katG-gyrB-A1-sp   P1-sp-L1      2153871            18     59.81   
1      katG-gyrA-A2-sp   P2-sp-L1      2154365            20     59.82   
2      katG-fgd1-A3-sp   P3-sp-L1      2155277            18     60.52   
3     mmpS5-mshA-A4-sp   P4-sp-L1       778851            20     59.96   
4      rpoB-ccsA-A1-ns   P1-ns-L1       760602            20     60.11   
5      pncA-rpoB-A2-ns   P2-ns-L1      2288500            20     59.76   
6      embB-rpoC-A3-ns   P3-ns-L4      4247277            18     61.20   
7     rpsL-mmpL5-A4-ns   P4-ns-L1       781291            20     61.07   
8    fabG1-mmpS5-A5-ns   P5-ns-L1      1673325            20     59.84   
9    fadA5-mmpR5-A6-ns   P6-ns-L1      3986666            19     60.52   
10        _-rpsL-A7-ns   P7-ns-L1      1475826            20     59.96   
11     gyrB-rplC-A8-ns   P8-ns-L1         7061            20     59.96   
12        _-fbiC-A9-ns   P9-ns-L1      1472666            20     59.81   
13  gid-Rv1258c-A10-ns  P10-ns-L1      4407538            18     60.20   
14    rplC-embR-A11-ns  P11-ns-L1       800907            19     60.38   
15       _-atpE-A12-ns  P12-ns-L1      1472057            20     60.39   
16      eis-rrs-A13-ns  P13-ns-L1      2715069            18     62.07   
17     ethA-rrl-A14-ns  P14-ns-L1      4326166            20     60.03   
18   ethA-fabG1-A15-ns  P15-ns-L1      4327288            20     59.83   
19    embC-inhA-A16-ns  P16-ns-L1      4242920            20     59.75   
20    inhA-rpsA-A17-ns  P17-ns-L1      1674257            20     60.05   
21    ethA-tlyA-A18-ns  P18-ns-L1      4326794            19     60.15   
22     ahpC-ndh-A19-ns  P19-ns-L1      2725909            19     60.15   
23    folC-katG-A20-ns  P20-ns-L1      2747076            18     60.43   
24    gyrB-gyrB-A21-ns  P21-ns-L1         6458            20     60.25   
25    embB-gyrA-A22-ns  P22-ns-L1      4249264            20     60.25   
26    rraA-fgd1-A23-ns  P23-ns-L1      4325727            20     60.46   
27     alr-mshA-A24-ns  P24-ns-L1      3841060            20     65.09   
28    thyX-ccsA-A25-ns  P25-ns-L1      3067730            20     60.11   
29    tlyA-rpoB-A26-ns  P26-ns-L1      1918307            19     60.45   
30    thyA-rpoC-A27-ns  P27-ns-L1      3073706            20     59.69   
31   rpoB-mmpL5-A28-ns  P28-ns-L1       761907            20     60.59   
32   thyA-mmpS5-A29-ns  P29-ns-L1      3074136            20     59.82   

     pLeft_GC       pLeft_Sequences  pLeft_EndStability  pRight_ID  \
0   61.111111    GGGCCGATCAACCCGAAT                3.34   P1-sp-R1   
1   50.000000  TTCCACGTCGGTTTGTTCCT                3.36   P2-sp-R1   
2   66.666667    GGCGCCATGGGTCTTACC                2.85   P3-sp-R1   
3   55.000000  GACCACAACCAGGATGAGCA                4.26   P4-sp-R1   
4   55.000000  AGGCGCTGTTGGACATCTAC                2.59   P1-ns-R1   
5   55.000000  TGAAGGTGTCGTAGAAGRGG                5.52   P2-ns-R1   
6   66.666667    GTTCCGGCCTGCATCGTC                4.20   P3-ns-R4   
7   60.000000  GTGGTACCGCYTCAACGTAG                3.51   P4-ns-R1   
8   55.000000  CAAACGTCACGAGRGTAACC                2.85   P5-ns-R1   
9   57.894737   CTCGCGCTGATCACCATGT                3.21   P6-ns-R1   
10  50.000000  AGCGAAATTCCTTGTCGGGT                5.28   P7-ns-R1   
11  55.000000  AGCGGTACAAGGGTCTAGGT                3.08   P8-ns-R1   
12  55.000000  TGTGGGTTTCCTTCCTTGGG                4.12   P9-ns-R1   
13  66.666667    CTCCACTCGCCATCCGTG                4.94  P10-ns-R1   
14  57.894737   AACGTGGTAACCCGCATCC                3.51  P11-ns-R1   
15  55.000000  TTTAGCGGTGTGGGATGAGC                4.26  P12-ns-R1   
16  66.666667    GCCACCGCGACGAAACTG                3.16  P13-ns-R1   
17  60.000000  GGGGTGAACTCCATGAAGGG                3.95  P14-ns-R1   
18  55.000000  AGCGTGTACATGTCGGAGTC                3.36  P15-ns-R1   
19  55.000000  CCAACTCACCGGTGATGGAT                3.41  P16-ns-R1   
20  55.000000  GTCGARYGC

In [51]:
gene_ = []

for i, row in accepted_primers.iterrows():
    for x, row_ in GF.iterrows():
        _ = False
        # print(row_)
        if (row['pLeft_coord'] < row_.iloc[4]) and (row['pRight_coord'] > row_.iloc[3]):
            _ = row_.iloc[1]
            # print(row)
            print(_)
            # print(row_.iloc[1])

            print(row_.iloc[3], row_.iloc[4])
            print(row['Amplicon_ID'], row['pLeft_coord'], row['pRight_coord'])
            if _:
                gene_.append(_)
            else:
                gene_.append('-')
            break
        
# accepted_primers['Amplicon_ID'] = [str(g) + '-' + str(a) for g, a in zip(gene_, accepted_primers['Amplicon_ID'])]
new_amplicon_id = [str(g) + '-' + str(a) for g, a in zip(gene_, accepted_primers['Amplicon_ID'])]


katG
2153889 2156111
gyrB-A1-sp 2153871 2154429
katG
2153889 2156111
gyrA-A2-sp 2154365 2155295
katG
2153889 2156111
fgd1-A3-sp 2155277 2156154
mmpS5
778477 778905
mshA-A4-sp 778851 779604
rpoB
759807 763325
ccsA-A1-ns 760602 761526
pncA
2288681 2289241
rpoB-A2-ns 2288500 2289343
embB
4246514 4249810
rpoC-A3-ns 4247277 4248109
rpsL
781560 781934
mmpL5-A4-ns 781291 782142
fabG1
1673440 1674183
mmpS5-A5-ns 1673325 1674124
fadA5
3985557 3986732
mmpR5-A6-ns 3986666 3987432
gyrB
5240 7267
rplC-A8-ns 7061 7776
gid
4407528 4408202
Rv1258c-A10-ns 4407538 4408212
rplC
800809 801462
embR-A11-ns 800907 801591
eis
2714124 2715332
rrs-A13-ns 2715069 2715700
ethA
4326004 4327473
rrl-A14-ns 4326166 4326844
ethA
4326004 4327473
fabG1-A15-ns 4327288 4327901
embC
4239863 4243147
inhA-A16-ns 4242920 4243538
inhA
1674202 1675011
rpsA-A17-ns 1674257 1674858
ethA
4326004 4327473
tlyA-A18-ns 4326794 4327313
ahpC
2726193 2726780
ndh-A19-ns 2725909 2726457
folC
2746135 2747598
katG-A20-ns 2747076 2747610
gyrB


In [35]:
new_amplicon_id 

['katG-gyrB-A1-sp',
 'katG-gyrA-A2-sp',
 'katG-fgd1-A3-sp',
 'furA-mshA-A4-sp',
 'mmpS5-ccsA-A1-ns',
 'rpoB-rpoB-A2-ns',
 'embB-rpoC-A3-ns',
 'rpsG-mmpL5-A4-ns',
 'fabG1-mmpS5-A5-ns',
 'fadA5-mmpR5-A6-ns',
 'gyrB-rpsL-A7-ns',
 'gyrA-rplC-A8-ns',
 'gid-fbiC-A9-ns',
 'rplC-Rv1258c-A10-ns',
 'rplD-embR-A11-ns',
 'eis-atpE-A12-ns',
 'ethA-rrs-A13-ns',
 'ethA-rrl-A14-ns',
 'ethR-fabG1-A15-ns',
 'embC-inhA-A16-ns',
 'embA-rpsA-A17-ns',
 'inhA-tlyA-A18-ns',
 'ethA-ndh-A19-ns',
 'ahpC-katG-A20-ns',
 'folC-gyrB-A21-ns',
 'valS-gyrA-A22-ns',
 'gyrB-fgd1-A23-ns',
 'embB-mshA-A24-ns',
 'rraA-ccsA-A25-ns',
 'ethA-rpoB-A26-ns',
 'alr-rpoC-A27-ns',
 'thyX-mmpL5-A28-ns',
 'tlyA-mmpS5-A29-ns']

In [30]:
new_amplicon_id 

['2153889-gyrB-A1-sp',
 '2153889-gyrA-A2-sp',
 '2153889-fgd1-A3-sp',
 '2156149-mshA-A4-sp',
 '778477-ccsA-A1-ns',
 '759807-rpoB-A2-ns',
 '4246514-rpoC-A3-ns',
 '781934-mmpL5-A4-ns',
 '1673440-mmpS5-A5-ns',
 '3985557-mmpR5-A6-ns',
 '5240-rpsL-A7-ns',
 '7302-rplC-A8-ns',
 '4407528-fbiC-A9-ns',
 '800809-Rv1258c-A10-ns',
 '801462-embR-A11-ns',
 '2714124-atpE-A12-ns',
 '4326004-rrs-A13-ns',
 '4326004-rrl-A14-ns',
 '4327549-fabG1-A15-ns',
 '4239863-inhA-A16-ns',
 '4243233-rpsA-A17-ns',
 '1674202-tlyA-A18-ns',
 '4326004-ndh-A19-ns',
 '2726193-katG-A20-ns',
 '2746135-gyrB-A21-ns',
 '2747595-gyrA-A22-ns',
 '5240-fgd1-A23-ns',
 '4246514-mshA-A24-ns',
 '4325495-ccsA-A25-ns',
 '4326004-rpoB-A26-ns',
 '3840194-rpoC-A27-ns',
 '3067193-mmpL5-A28-ns',
 '1917940-mmpS5-A29-ns']

In [30]:
GF[GF['gene_name'] == 'katG']

gene_id gene_name         chr    start      end
7987  Rv1908c      katG  Chromosome  2153889  2156111

In [7]:
def complement_sequence(dna_sequence):
    trans = str.maketrans('ATCGRYSWKMNBVDH-.', 'TAGCYRWSMKNVBHD-.')
    return dna_sequence.upper().translate(trans)

In [26]:
def check_heterodimer(seq1, seq2, length=5):
    """Checks if the last 'length' bases of seq1 are found anywhere in seq2."""
    # Get the last 'length' bases of seq1
    last_bases_seq1 = seq1[-length:]
    last_bases_seq1_com = complement_sequence(last_bases_seq1)
    last_bases_seq1_rev_com = last_bases_seq1_com[::-1]
    # Check if these bases are found anywhere in seq2
    is_binding_com = last_bases_seq1_com in seq2
    is_binding_rev_com = last_bases_seq1_rev_com in seq2
    is_binding = is_binding_com or is_binding_rev_com
    
    if is_binding == False:
        last_bases_seq1 = seq2[-length:]
        last_bases_seq1_com = complement_sequence(last_bases_seq1)
        last_bases_seq1_rev_com = last_bases_seq1_com[::-1]
        # Check if these bases are found anywhere in seq2
        is_binding_com = last_bases_seq1_com in seq2
        is_binding_rev_com = last_bases_seq1_rev_com in seq2
        is_binding = is_binding_com or is_binding_rev_com
    
    return is_binding

# Example sequences
seq1 = "ATGCGTACGTAGCTAGCTAG"
seq2 = "GCTAGCTAGCGTACGTAGCT"

# Check for heterodimer binding using the last 4 bases
binding_result = check_heterodimer(seq1, seq2, length=4)

# Output the result
print(f"Heterodimer binding found: {binding_result}")


False True
True
Heterodimer binding found: True


In [17]:
        # mv_conc: 50
        # dv_conc: 
        # dntp_conc: dNTP conc. (mM)
        # dna_conc: DNA conc. (nM)
        # temp_c: Simulation temperature for dG (Celsius)
        # max_loop: Maximum size of loops in the structure
        # output_structure: If `True`, the ASCII dimer structure is saved

calc_heterodimer('GGGCCGATCAACCCGAAT','TTCCACGTCGGTTTGTTCCT', mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False)


ThermoResult(structure_found=True, tm=-4.17, dg=-4296.50, dh=-45200.00, ds=-131.88)

In [7]:
calc_heterodimer('GGCGCCATGGGTCTTACC','MCGTGATCACAGCCCGATAA')

ThermoResult(structure_found=True, tm=-24.44, dg=-2964.57, dh=-33400.00, ds=-98.13)

In [43]:
for i, row in df.iterrows():
    # calc_heterodimer
    # calc_heterodimer(row['pLeft_Sequences'],row['pRight_Sequences'])
    for i_, row_ in df.iterrows():
        print('')
        print(calc_heterodimer(row['pLeft_Sequences'],row_['pLeft_Sequences'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm)
        print(calc_heterodimer(row['pRight_Sequences'],row_['pLeft_Sequences'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False))
        print(calc_heterodimer(row['pLeft_Sequences'],row_['pRight_Sequences'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False))
        print(calc_heterodimer(row['pRight_Sequences'],row_['pRight_Sequences'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False))
        break
    break


4.011169325327671
ThermoResult(structure_found=True, tm=0.33, dg=-4538.01, dh=-49800.00, ds=-145.94)
ThermoResult(structure_found=True, tm=0.33, dg=-4538.01, dh=-49800.00, ds=-145.94)
ThermoResult(structure_found=True, tm=2.29, dg=-4510.85, dh=-53200.00, ds=-156.99)


In [61]:
dimer_pairs = []
for i, row in df.iterrows():
    # calc_heterodimer
    # calc_heterodimer(row['pLeft_ID'],row['pRight_ID'])
    for i_, row_ in df.iterrows():
        if calc_heterodimer(row['pLeft_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).structure_found == True:
            # print(row['pLeft_ID'],row_['pLeft_ID'])
            dimer_pairs.append(f"{row['pLeft_ID']} with {row_['pLeft_ID']}")

            
        if calc_heterodimer(row['pRight_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).structure_found == True:
            # print(row['pRight_ID'],row_['pLeft_ID'])
            dimer_pairs.append(f"{row['pRight_ID']} with {row_['pLeft_ID']}")

            
        if calc_heterodimer(row['pLeft_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).structure_found == True:
            # print(row['pLeft_ID'],row_['pRight_ID'])
            dimer_pairs.append(f"{row['pLeft_ID']} with {row_['pRight_ID']}")

            
        if calc_heterodimer(row['pRight_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).structure_found == True:
            # print(row['pRight_ID'],row_['pRight_ID'])
            dimer_pairs.append(f"{row['pRight_ID']} with {row_['pRight_ID']}")

            

In [ ]:
dimer_pairs

In [ ]:
for i, row in df.iterrows():
    # calc_heterodimer
    # calc_heterodimer(row['pLeft_ID'],row['pRight_ID'])
    for i_, row_ in df.iterrows():
        print(calc_heterodimer(row['pLeft_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False))
        if calc_heterodimer(row['pLeft_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm >1:
            print(row['pLeft_ID'],row_['pLeft_ID'])
        if calc_heterodimer(row['pRight_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm >1:
            print(row['pRight_ID'],row_['pLeft_ID'])
        if calc_heterodimer(row['pLeft_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm >1:
            print(row['pLeft_ID'],row_['pRight_ID'])
        if calc_heterodimer(row['pRight_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm >1:
            print(row['pRight_ID'],row_['pRight_ID'])

In [75]:
for i, row in df.iterrows():
    # calc_heterodimer
    # calc_heterodimer(row['pLeft_ID'],row['pRight_ID'])
    for i_, row_ in df.iterrows():
        _ = calc_heterodimer(row['pLeft_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False)
        if _.tm > 50 or _.dg < -1:
            print(row['pLeft_ID'],row_['pLeft_ID'])
            print(_)
        _ = calc_heterodimer(row['pRight_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False)
        if _.tm > 50 or _.dg < -1:
            print(row['pRight_ID'],row_['pLeft_ID'])
            print(_)
        _ = calc_heterodimer(row['pLeft_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False)
        if _.tm > 50 or _.dg < -1:
            print(row['pLeft_ID'],row_['pRight_ID'])
            print(_)
        _ = calc_heterodimer(row['pRight_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False)
        if _.tm > 50 or _.dg < -1:
            print(row['pRight_ID'],row_['pRight_ID'])
            print(_)
    #     break
    # break

P21-ns-L1 P21-ns-L1
ThermoResult(structure_found=True, tm=-129.78, dg=-48.26, dh=-9600.00, ds=-30.80)
P21-ns-R1 P21-ns-L1
ThermoResult(structure_found=True, tm=-129.78, dg=-48.26, dh=-9600.00, ds=-30.80)
P21-ns-L1 P21-ns-R1
ThermoResult(structure_found=True, tm=-129.78, dg=-48.26, dh=-9600.00, ds=-30.80)
P21-ns-R1 P21-ns-R1
ThermoResult(structure_found=True, tm=-129.78, dg=-48.26, dh=-9600.00, ds=-30.80)


In [ ]:
for i, row in df.iterrows():
    # calc_heterodimer
    # calc_heterodimer(row['pLeft_ID'],row['pRight_ID'])
    for i_, row_ in df.iterrows():
        print('')
        if calc_heterodimer(row['pLeft_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm > 50:
            print(row['pLeft_ID'],row_['pLeft_ID'])
        if calc_heterodimer(row['pRight_ID'],row_['pLeft_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm > 50:
            print(row['pRight_ID'],row_['pLeft_ID'])
        if calc_heterodimer(row['pLeft_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm > 50:
            print(row['pLeft_ID'],row_['pRight_ID'])
        if calc_heterodimer(row['pRight_ID'],row_['pRight_ID'], mv_conc=50, dv_conc=1.5, dntp_conc=0.8, dna_conc=50, temp_c=37, max_loop=30, output_structure=False).tm > 50:
            print(row['pRight_ID'],row_['pRight_ID'])
    #     break
    # break

In [67]:
dimer_pairs 

['P1-sp-L1 with P2-sp-L1',
 'P1-sp-R1 with P2-sp-L1',
 'P1-sp-L1 with P2-sp-R1',
 'P1-sp-R1 with P2-sp-R1',
 'P1-sp-L1 with P2-ns-L1',
 'P1-sp-R1 with P2-ns-L1',
 'P1-sp-L1 with P2-ns-R1',
 'P1-sp-R1 with P2-ns-R1',
 'P1-sp-L1 with P12-ns-L1',
 'P1-sp-R1 with P12-ns-L1',
 'P1-sp-L1 with P12-ns-R1',
 'P1-sp-R1 with P12-ns-R1',
 'P1-sp-L1 with P20-ns-L1',
 'P1-sp-R1 with P20-ns-L1',
 'P1-sp-L1 with P20-ns-R1',
 'P1-sp-R1 with P20-ns-R1',
 'P1-sp-L1 with P21-ns-L1',
 'P1-sp-R1 with P21-ns-L1',
 'P1-sp-L1 with P21-ns-R1',
 'P1-sp-R1 with P21-ns-R1',
 'P1-sp-L1 with P22-ns-L1',
 'P1-sp-R1 with P22-ns-L1',
 'P1-sp-L1 with P22-ns-R1',
 'P1-sp-R1 with P22-ns-R1',
 'P1-sp-L1 with P23-ns-L1',
 'P1-sp-R1 with P23-ns-L1',
 'P1-sp-L1 with P23-ns-R1',
 'P1-sp-R1 with P23-ns-R1',
 'P1-sp-L1 with P24-ns-L1',
 'P1-sp-R1 with P24-ns-L1',
 'P1-sp-L1 with P24-ns-R1',
 'P1-sp-R1 with P24-ns-R1',
 'P1-sp-L1 with P25-ns-L1',
 'P1-sp-R1 with P25-ns-L1',
 'P1-sp-L1 with P25-ns-R1',
 'P1-sp-R1 with P25-ns-R1',


In [63]:
df_left  =df[['pLeft_ID','pLeft_Sequences']]
df_right = df[['pRight_ID','pRight_Sequences']]
df_left.columns = ['Amplicon_ID','pSequences']
df_right.columns = ['Amplicon_ID','pSequences']

In [64]:
pd.concat([df_left, df_right], axis=0).to_csv('primer_pairs.csv', index=False, sep='\t')

In [65]:
input_text = """             Self-Dimers:

1 dimer for: P4-ns-L1
5-gtggtaccgcytcaacgtag->
              | |||| |
            <-gatgcaactycgccatggtg-5

1 dimer for: P16-ns-L1
5-ccaactcaccggtgatggat->
       ||||||||||     
<-taggtagtggccactcaacc-5

1 dimer for: P19-ns-L1
     5-gcttgccggaaagacatgc->
        ||| |||| ||| 
<-cgtacagaaaggccgttcg-5

1 dimer for: P25-ns-L1
5-accgagaaatgtccgacgtc->
                ||||||
              <-ctgcagcctgtaaagagcca-5

1 dimer for: P2-sp-R1
5-ggtaagacccatggcgcc->
              ||||||
            <-ccgcggtacccagaatgg-5

1 dimer for: P2-ns-R1
5-tggtcatgttcgcgatcgtc->
       | | |||||| | | 
    <-ctgctagcgcttgtactggt-5

1 dimer for: P4-ns-R1
       5-ttgacattgtcgagagcccg->
         | |||| |||| |
<-gcccgagagctgttacagtt-5

1 dimer for: P6-ns-R1
5-agcgacctgttggccaag->
           |||||||| 
        <-gaaccggttgtccagcga-5

1 dimer for: P11-ns-R1
5-ccacctggtgcatcagcg->
      ||| |||| |||  
  <-gcgactacgtggtccacc-5

1 dimer for: P27-ns-R1
5-gatttgctgcgcaccgatc->
     | | |||||| | |  
 <-ctagccacgcgtcgtttag-5


               Cross Primer Dimers:

P1-sp-L1 with P2-sp-L1
P1-sp-L1
5-ttccacgtcggtttgttcct->
         |||| ||| ||  
     <-taagcccaactagccggg-5

P1-sp-L1 with P6-ns-L1
P1-sp-L1
 5-ctcgcgctgatcaccatgt->
    ||| | |||||  |  
<-taagcccaactagccggg-5

P1-sp-L1 with P2-ns-R1
P1-sp-L1
5-tggtcatgttcgcgatcgtc->
          |||| | |  ||
       <-taagcccaactagccggg-5

P1-sp-L1 with P13-ns-R1
P1-sp-L1
   5-ygtgatgatcgaccgggttt->
      | | |||||| || 
<-taagcccaactagccggg-5

P2-sp-L1 with P4-ns-L1
P2-sp-L1
5-ttccacgtcggtttgttcct->
    | |||| |    | | | 
  <-gatgcaactycgccatggtg-5

P2-sp-L1 with P25-ns-L1
P2-sp-L1
5-ttccacgtcggtttgttcct->
      |||||||    |||  
   <-ctgcagcctgtaaagagcca-5

P3-sp-L1 with P19-ns-L1
P3-sp-L1
5-gcttgccggaaagacatgc->
         || |||||    
       <-ccattctgggtaccgcgg-5

P3-sp-L1 with P2-sp-R1
P3-sp-L1
5-ggcgccatgggtcttacc->
  ||||||||||||||||||
<-ccgcggtacccagaatgg-5

P4-sp-L1 with P7-ns-L1
P4-sp-L1
5-gaccacaaccaggatgagca->
     | | || ||||    | 
  <-tgggctgttccttaaagcga-5

P4-sp-L1 with P11-ns-L1
P4-sp-L1
5-gaccacaaccaggatgagca->
             ||||| |  
           <-cctacgcccaatggtgcaa-5

P4-sp-L1 with P17-ns-L1
P4-sp-L1
5-gaccacaaccaggatgagca->
              |||| | |
           <-gctacactttgcgyragctg-5

P4-sp-L1 with P29-ns-L1
P4-sp-L1
5-gaccacaaccaggatgagca->
              | ||||||
            <-ccactcgtgtagctggtcta-5

P4-sp-L1 with P13-ns-R1
P4-sp-L1
5-gaccacaaccaggatgagca->
        |||| ||  || ||
     <-tttgggccagctagtagtgy-5

P4-sp-L1 with P17-ns-R1
P4-sp-L1
5-gaccacaaccaggatgagca->
    ||| | ||||       |
  <-ggtctaggtcgacgagctc-5

P1-ns-L1 with P25-ns-L1
P1-ns-L1
5-aggcgctgttggacatctac->
      |  || |||||| |  
    <-ctgcagcctgtaaagagcca-5

P1-ns-L1 with P27-ns-L1
P1-ns-L1
5-aggcgctgttggacatctac->
              | | ||||
            <-tcttgatgctaggcgtaggc-5

P1-ns-L1 with P6-ns-R1
P1-ns-L1
5-aggcgctgttggacatctac->
          |||| || |   
       <-gaaccggttgtccagcga-5

P1-ns-L1 with P7-ns-R1
P1-ns-L1
5-aggcgctgttggacatctac->
           |||| | | ||
        <-gacctatcggtgcaagccag-5

P1-ns-L1 with P11-ns-R1
P1-ns-L1
5-aggcgctgttggacatctac->
     ||||| || ||      
   <-gcgactacgtggtccacc-5

P1-ns-L1 with P18-ns-R1
P1-ns-L1
5-aggcgctgttggacatctac->
            | |||| |  
          <-catgtacagcctcagcctcg-5

P1-ns-L1 with P23-ns-R1
P1-ns-L1
5-aggcgctgttggacatctac->
       || || |||||||  
     <-gagaagctgtagactcgggc-5

P3-ns-L4 with P24-ns-L1
P3-ns-L4
5-gaagtcgatgccgggcggat->
     | |||||| || |||| 
   <-ctgctacgtccggccttg-5

P3-ns-L4 with P26-ns-L1
P3-ns-L4
5-caacgatcctcgggtggtg->
    ||||| |  |   |   
 <-ctgctacgtccggccttg-5

P3-ns-L4 with P28-ns-L1
P3-ns-L4
5-gtactgacgacggcgagatg->
       ||||| |  |    |
     <-ctgctacgtccggccttg-5

P3-ns-L4 with P29-ns-L1
P3-ns-L4
5-atctggtcgatgtgctcacc->
       | |||||    |   
     <-ctgctacgtccggccttg-5

P3-ns-L4 with P28-ns-R1
P3-ns-L4
5-ttctcaccgaagatggcacg->
          || ||||     
        <-ctgctacgtccggccttg-5

P4-ns-L1 with P12-ns-R1
P4-ns-L1
5-gtggtaccgcytcaacgtag->
             |||| | | 
           <-agtttccttaactrcccccg-5

P5-ns-L1 with P29-ns-R1
P5-ns-L1
5-caaacgtcacgagrgtaacc->
        ||||||  |  || 
      <-agtgctaacygtgcgccag-5

P6-ns-L1 with P11-ns-L1
P6-ns-L1
5-ctcgcgctgatcaccatgt->
       || | | |||| ||
 <-cctacgcccaatggtgcaa-5

P6-ns-L1 with P1-sp-R1
P6-ns-L1
5-caacatcacggtgcccttca->
        || ||||  |  | 
     <-tgtaccactagtcgcgctc-5

P6-ns-L1 with P4-sp-R1
P6-ns-L1
5-acaaaaacggtgaccccaca->
      | | ||||| |     
    <-tgtaccactagtcgcgctc-5

P6-ns-L1 with P4-ns-R1
P6-ns-L1
5-ttgacattgtcgagagcccg->
     |||| ||    ||| ||
   <-tgtaccactagtcgcgctc-5

P6-ns-L1 with P11-ns-R1
P6-ns-L1
5-ctcgcgctgatcaccatgt->
      |||||||   |  | 
    <-gcgactacgtggtccacc-5

P6-ns-L1 with P14-ns-R1
P6-ns-L1
5-ctacatcgtgtcgcagccag->
    |||| |||       |  
  <-tgtaccactagtcgcgctc-5

P6-ns-L1 with P29-ns-R1
P6-ns-L1
5-ctcgcgctgatcaccatgt->
            |||| ||  
          <-agtgctaacygtgcgccag-5

P7-ns-L1 with P3-sp-R1
P7-ns-L1
5-agcgaaattccttgtcgggt->
             || ||||| 
           <-aatagcccgacactagtgcm-5

P7-ns-L1 with P4-sp-R1
P7-ns-L1
5-agcgaaattccttgtcgggt->
              ||| ||||
            <-acaccccagtggcaaaaaca-5

P7-ns-L1 with P5-ns-R1
P7-ns-L1
5-agcgaaattccttgtcgggt->
         ||||| |      
      <-gaaggaccgaaggctcctac-5

P7-ns-L1 with P18-ns-R1
P7-ns-L1
5-agcgaaattccttgtcgggt->
          | | ||||||  
       <-catgtacagcctcagcctcg-5

P7-ns-L1 with P20-ns-R1
P7-ns-L1
5-agcgaaattccttgtcgggt->
       ||||| |||  | | 
     <-ttaagtaacgtcgcgtcgg-5

P8-ns-L1 with P18-ns-R1
P8-ns-L1
5-agcggtacaagggtctaggt->
      ||||| |         
    <-catgtacagcctcagcctcg-5

P8-ns-L1 with P26-ns-R1
P8-ns-L1
5-agcggtacaagggtctaggt->
         ||||||  | |  
       <-gttccctaacctcctacgcc-5

P9-ns-L1 with P11-ns-L1
P9-ns-L1
   5-tgtgggtttccttccttggg->
     || ||||| ||    |
<-cctacgcccaatggtgcaa-5

P9-ns-L1 with P14-ns-L1
P9-ns-L1
5-tgtgggtttccttccttggg->
           ||||| | | |
        <-gggaagtacctcaagtgggg-5

P9-ns-L1 with P5-ns-R1
P9-ns-L1
5-tgtgggtttccttccttggg->
            |||||| |  
          <-gaaggaccgaaggctcctac-5

P10-ns-L1 with P24-ns-L1
P10-ns-L1
5-gaagtcgatgccgggcggat->
        |||| || | ||| 
  <-gtgcctaccgctcacctc-5

P10-ns-L1 with P5-ns-R1
P10-ns-L1
5-catcctcggaagccaggaag->
      | |||| | |  |  |
    <-gtgcctaccgctcacctc-5

P10-ns-L1 with P8-ns-R1
P10-ns-L1
5-ggccgtcgtagttagggatg->
               | |||||
            <-gtgcctaccgctcacctc-5

P10-ns-L1 with P23-ns-R1
P10-ns-L1
5-cgggctcagatgtcgaagag->
      | | |||| |||   |
    <-gtgcctaccgctcacctc-5

P11-ns-L1 with P12-ns-L1
P11-ns-L1
5-tttagcggtgtgggatgagc->
              ||||| | 
            <-cctacgcccaatggtgcaa-5

P11-ns-L1 with P24-ns-L1
P11-ns-L1
5-gaagtcgatgccgggcggat->
        ||||| ||      
     <-cctacgcccaatggtgcaa-5

P11-ns-L1 with P27-ns-L1
P11-ns-L1
5-cggatgcggatcgtagttct->
   |||||||| |        |
 <-cctacgcccaatggtgcaa-5

P11-ns-L1 with P3-ns-R4
P11-ns-L1
5-ggtgggcaggatgrggtagt->
          ||||| ||    
        <-cctacgcccaatggtgcaa-5

P12-ns-L1 with P17-ns-L1
P12-ns-L1
5-tttagcggtgtgggatgagc->
       || ||||  | |   
     <-gctacactttgcgyragctg-5

P12-ns-L1 with P29-ns-L1
P12-ns-L1
5-tttagcggtgtgggatgagc->
               | |||||
             <-ccactcgtgtagctggtcta-5

P12-ns-L1 with P4-sp-R1
P12-ns-L1
5-tttagcggtgtgggatgagc->
          |||||| | | |
        <-acaccccagtggcaaaaaca-5

P12-ns-L1 with P16-ns-R1
P12-ns-L1
5-tttagcggtgtgggatgagc->
      | || ||||     | 
    <-caccgcacctatggccattc-5

P12-ns-L1 with P27-ns-R1
P12-ns-L1
5-tttagcggtgtgggatgagc->
     | ||||| |     |  
  <-ctagccacgcgtcgtttag-5

P13-ns-L1 with P23-ns-L1
P13-ns-L1
5-tgatgtcatcgccgagttgg->
     | || |||| | |  
<-gtcaaagcagcgccaccg-5

P13-ns-L1 with P1-sp-R1
P13-ns-L1
   5-caacatcacggtgcccttca->
        | || ||||| |
<-gtcaaagcagcgccaccg-5

P14-ns-L1 with P16-ns-L1
P14-ns-L1
5-ggggtgaactccatgaaggg->
           ||||| |  ||
        <-taggtagtggccactcaacc-5

P14-ns-L1 with P27-ns-L1
P14-ns-L1
5-ggggtgaactccatgaaggg->
       ||||| |     |  
    <-tcttgatgctaggcgtaggc-5

P14-ns-L1 with P29-ns-L1
P14-ns-L1
5-ggggtgaactccatgaaggg->
    ||||| | |   ||   |
  <-ccactcgtgtagctggtcta-5

P14-ns-L1 with P1-sp-R1
P14-ns-L1
5-ggggtgaactccatgaaggg->
               |||||||
             <-acttcccgtggcactacaac-5

P14-ns-L1 with P1-ns-R1
P14-ns-L1
    5-ccatgtagtccacctcagacg->
       |||| ||| |||| |
<-gggaagtacctcaagtgggg-5

P14-ns-L1 with P25-ns-R1
P14-ns-L1
      5-ggggtgaactccatgaaggg->
        | | ||||| ||| 
<-gagggtctcaacttggggtg-5

P15-ns-L1 with P19-ns-L1
P15-ns-L1
5-agcgtgtacatgtcggagtc->
          ||||||      
       <-cgtacagaaaggccgttcg-5

P15-ns-L1 with P25-ns-L1
P15-ns-L1
5-agcgtgtacatgtcggagtc->
           | ||||||   
        <-ctgcagcctgtaaagagcca-5

P15-ns-L1 with P2-ns-R1
P15-ns-L1
    5-agcgtgtacatgtcggagtc->
      | || | ||||| |  
<-ctgctagcgcttgtactggt-5

P15-ns-L1 with P10-ns-R1
P15-ns-L1
5-agcgtgtacatgtcggagtc->
              | |||| |
            <-aycctctgtaatgcgtgagg-5

P15-ns-L1 with P18-ns-R1
P15-ns-L1
5-agcgtgtacatgtcggagtc->
       |||||||||||||||
     <-catgtacagcctcagcctcg-5

P15-ns-L1 with P24-ns-R1
P15-ns-L1
5-agcgtgtacatgtcggagtc->
             ||||| || 
           <-cagccgcactgacaggac-5

P16-ns-L1 with P1-ns-R1
P16-ns-L1
5-ccatgtagtccacctcagacg->
          |||| | | |  |
       <-taggtagtggccactcaacc-5

P16-ns-L1 with P11-ns-R1
P16-ns-L1
5-ccaactcaccggtgatggat->
           || ||||| | 
         <-gcgactacgtggtccacc-5

P16-ns-L1 with P27-ns-R1
P16-ns-L1
5-ccaactcaccggtgatggat->
         | |||||      
      <-ctagccacgcgtcgtttag-5

P18-ns-L1 with P23-ns-L1
P18-ns-L1
5-tgatgtcatcgccgagttgg->
         |||||||||  | 
       <-tagcggctcttcgagttgg-5

P18-ns-L1 with P4-ns-R1
P18-ns-L1
5-ttgacattgtcgagagcccg->
       || | |||||  |  
     <-tagcggctcttcgagttgg-5

P18-ns-L1 with P24-ns-R1
P18-ns-L1
5-ggttgagcttctcggcgat->
             ||||||  
          <-cagccgcactgacaggac-5

P18-ns-L1 with P28-ns-R1
P18-ns-L1
5-ttctcaccgaagatggcacg->
     || ||||  |   ||  
  <-tagcggctcttcgagttgg-5

P19-ns-L1 with P15-ns-R1
P19-ns-L1
5-ttgtgcgacccgaatgtctc->
             | |||||| 
           <-cgtacagaaaggccgttcg-5

P20-ns-L1 with P7-ns-R1
P20-ns-L1
5-gaccgaacgtggctatccag->
        |||||||  |    
     <-ctgcaccgtcaccagctc-5

P20-ns-L1 with P29-ns-R1
P20-ns-L1
5-gaccgcgtgycaatcgtga->
       |||| || | ||  
   <-ctgcaccgtcaccagctc-5

P22-ns-L1 with P2-ns-R1
P22-ns-L1
5-cccgacgacctatacggaga->
     ||||| |   |    ||
   <-ctgctagcgcttgtactggt-5

P23-ns-L1 with P29-ns-L1
P23-ns-L1
  5-tgatgtcatcgccgagttgg->
    |||   ||||| | || |
<-ccactcgtgtagctggtcta-5

P23-ns-L1 with P22-ns-R1
P23-ns-L1
5-tgatgtcatcgccgagttgg->
              | |||| |
           <-cgttcaasctgtgggacca-5

P23-ns-L1 with P25-ns-R1
P23-ns-L1
5-tgatgtcatcgccgagttgg->
          || | |||||| 
       <-gagggtctcaacttggggtg-5

P24-ns-L1 with P16-ns-R1
P24-ns-L1
  5-gaagtcgatgccgggcggat->
    |  || ||| ||||   |
<-caccgcacctatggccattc-5

P25-ns-L1 with P15-ns-R1
P25-ns-L1
5-accgagaaatgtccgacgtc->
     |||| ||    | ||  
   <-ctctgtaagcccagcgtgtt-5

P25-ns-L1 with P23-ns-R1
P25-ns-L1
5-accgagaaatgtccgacgtc->
           | | |||| ||
        <-gagaagctgtagactcgggc-5

P26-ns-L1 with P27-ns-L1
P26-ns-L1
   5-cggatgcggatcgtagttct->
     |    | ||||||| |
<-gtggtgggctcctagcaac-5

P28-ns-L1 with P24-ns-R1
P28-ns-L1
5-gtactgacgacggcgagatg->
          | ||||| ||  
        <-cagccgcactgacaggac-5

P29-ns-L1 with P3-sp-R1
P29-ns-L1
5-atctggtcgatgtgctcacc->
   |      | |||| |||| 
 <-aatagcccgacactagtgcm-5

P29-ns-L1 with P4-ns-R1
P29-ns-L1
5-atctggtcgatgtgctcacc->
             | |||| | 
          <-gcccgagagctgttacagtt-5

    5-atctggtcgatgtgctcacc->
       ||| | | ||||   
<-gcccgagagctgttacagtt-5

P29-ns-L1 with P13-ns-R1
P29-ns-L1
  5-atctggtcgatgtgctcacc->
    | | |||||||   | | 
<-tttgggccagctagtagtgy-5

P1-sp-R1 with P13-ns-R1
P1-sp-R1
5-caacatcacggtgcccttca->
      | | ||||        
   <-tttgggccagctagtagtgy-5

P1-sp-R1 with P28-ns-R1
P1-sp-R1
5-caacatcacggtgcccttca->
            ||||| |   
         <-gcacggtagaagccactctt-5

P1-sp-R1 with P29-ns-R1
P1-sp-R1
5-caacatcacggtgcccttca->
       ||||| |   |    
     <-agtgctaacygtgcgccag-5

P3-sp-R1 with P13-ns-R1
P3-sp-R1
5-mcgtgatcacagcccgataa->
            | |||| |  
         <-tttgggccagctagtagtgy-5

P4-sp-R1 with P27-ns-R1
P4-sp-R1
5-acaaaaacggtgaccccaca->
       | |||||  |    |
    <-ctagccacgcgtcgtttag-5

P2-ns-R1 with P7-ns-R1
P2-ns-R1
     5-tggtcatgttcgcgatcgtc->
       | | || |||||   
<-gacctatcggtgcaagccag-5

P2-ns-R1 with P29-ns-R1
P2-ns-R1
5-tggtcatgttcgcgatcgtc->
           || |||| | |
         <-agtgctaacygtgcgccag-5

P3-ns-R4 with P26-ns-R1
P3-ns-R4
    5-ggtgggcaggatgrggtagt->
      ||   | |||||| ||
<-gttccctaacctcctacgcc-5

P3-ns-R4 with P27-ns-R1
P3-ns-R4
    5-ggtgggcaggatgrggtagt->
      |||| |||| |    
<-ctagccacgcgtcgtttag-5

P4-ns-R1 with P7-ns-R1
P4-ns-R1
5-ttgacattgtcgagagcccg->
           | || ||||  
        <-gacctatcggtgcaagccag-5

P6-ns-R1 with P7-ns-R1
P6-ns-R1
5-gaccgaacgtggctatccag->
         | |||| | |  |
       <-gaaccggttgtccagcga-5

P7-ns-R1 with P24-ns-R1
P7-ns-R1
5-gaccgaacgtggctatccag->
         |||| || ||| |
  <-cagccgcactgacaggac-5

P11-ns-R1 with P23-ns-R1
P11-ns-R1
5-cgggctcagatgtcgaagag->
      | | ||||    ||  
    <-gcgactacgtggtccacc-5

P12-ns-R1 with P20-ns-R1
P12-ns-R1
5-gcccccrtcaattcctttga->
           ||||| ||  |
         <-ttaagtaacgtcgcgtcgg-5

P14-ns-R1 with P15-ns-R1
P14-ns-R1
  5-ctacatcgtgtcgcagccag->
         ||| ||||||   
<-ctctgtaagcccagcgtgtt-5

P14-ns-R1 with P18-ns-R1
P14-ns-R1
5-ctacatcgtgtcgcagccag->
        | ||||| || | |
   <-catgtacagcctcagcctcg-5

P15-ns-R1 with P28-ns-R1
P15-ns-R1
5-ttgtgcgacccgaatgtctc->
    |||| | |          
 <-gcacggtagaagccactctt-5

P18-ns-R1 with P23-ns-R1
P18-ns-R1
5-gctccgactccgacatgtac->
         || |||||| |  
     <-gagaagctgtagactcgggc-5

P18-ns-R1 with P25-ns-R1
P18-ns-R1
5-gctccgactccgacatgtac->
         |||| | |  |  
       <-gagggtctcaacttggggtg-5

P19-ns-R1 with P25-ns-R1
P19-ns-R1
5-tgtcaatcyaaacccccagg->
              | ||||| 
            <-gagggtctcaacttggggtg-5

P20-ns-R1 with P27-ns-R1
P20-ns-R1
   5-ggctgcgctgcaatgaatt->
      | ||||| ||||   
<-ctagccacgcgtcgtttag-5
"""

In [66]:
lines = input_text.strip().split('\n')

# Filter lines that contain the word "with"
with_lines = [line for line in lines if 'with' in line]

# Print the resulting list of lines
print(with_lines)

['P1-sp-L1 with P2-sp-L1', 'P1-sp-L1 with P6-ns-L1', 'P1-sp-L1 with P2-ns-R1', 'P1-sp-L1 with P13-ns-R1', 'P2-sp-L1 with P4-ns-L1', 'P2-sp-L1 with P25-ns-L1', 'P3-sp-L1 with P19-ns-L1', 'P3-sp-L1 with P2-sp-R1', 'P4-sp-L1 with P7-ns-L1', 'P4-sp-L1 with P11-ns-L1', 'P4-sp-L1 with P17-ns-L1', 'P4-sp-L1 with P29-ns-L1', 'P4-sp-L1 with P13-ns-R1', 'P4-sp-L1 with P17-ns-R1', 'P1-ns-L1 with P25-ns-L1', 'P1-ns-L1 with P27-ns-L1', 'P1-ns-L1 with P6-ns-R1', 'P1-ns-L1 with P7-ns-R1', 'P1-ns-L1 with P11-ns-R1', 'P1-ns-L1 with P18-ns-R1', 'P1-ns-L1 with P23-ns-R1', 'P3-ns-L4 with P24-ns-L1', 'P3-ns-L4 with P26-ns-L1', 'P3-ns-L4 with P28-ns-L1', 'P3-ns-L4 with P29-ns-L1', 'P3-ns-L4 with P28-ns-R1', 'P4-ns-L1 with P12-ns-R1', 'P5-ns-L1 with P29-ns-R1', 'P6-ns-L1 with P11-ns-L1', 'P6-ns-L1 with P1-sp-R1', 'P6-ns-L1 with P4-sp-R1', 'P6-ns-L1 with P4-ns-R1', 'P6-ns-L1 with P11-ns-R1', 'P6-ns-L1 with P14-ns-R1', 'P6-ns-L1 with P29-ns-R1', 'P7-ns-L1 with P3-sp-R1', 'P7-ns-L1 with P4-sp-R1', 'P7-ns-L1 wit

In [162]:
v = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [167]:
v[v['type']!='transcript_ablation'].to_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv', index=False)

In [161]:
len('absdvsdv')

8

In [158]:
numbers = [10, 1, 2, 3, 10, 20]
for x in combinations(numbers, 1):
    print(x)

(10,)
(1,)
(2,)
(3,)
(10,)
(20,)


In [160]:
_ = [10, 1]
numbers = [i for i in numbers if i not in _]


In [159]:
numbers.sort()
print(numbers)

[1, 2, 3, 10, 10, 20]


In [141]:
accepted_primer = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output_seg/Amplicon_design_output/Primer_design-accepted_primers-4-700_800_100_2.csv')

In [153]:
# create a list
prime_numbers = [2, 3, 5, 7, 9, 9, 11]

# remove 9 from the list
prime_numbers.remove(9)
print(prime_numbers)

[2, 3, 5, 7, 9, 11]


In [144]:
accepted_primer

Amplicon_ID  pLeft_ID  pLeft_coord  pLeft_length  pLeft_Tm  pLeft_GC  \
0       A1-sp  P1-sp-L1       778940            20     60.11      55.0   
1       A1-ns  P1-ns-L1       760602            20     60.11      55.0   
2       A2-ns  P2-ns-L1      2155074            20     59.83      60.0   
3       A3-ns  P3-ns-L1      2288500            20     59.76      55.0   

        pLeft_Sequences  pLeft_EndStability pRight_ID  pRight_coord  \
0  TCTGGTGACGCATACCGAAC                3.95  P1-sp-R1        779819   
1  AGGCGCTGTTGGACATCTAC                2.59  P1-ns-R1        761526   
2  CTCTTCGTCAGCTCCCACTC                3.51  P2-ns-R1       2155867   
3  TGAAGGTGTCGTAGAAGRGG                5.52  P3-ns-R1       2289310   

   pRight_length  pRight_Tm  pRight_GC       pRight_Sequences  \
0             20      59.82  55.000000   CCAGATCGGTGACATGCTGA   
1             21      59.86  57.142857  CCATGTAGTCCACCTCAGACG   
2             20      60.04  55.000000   AAGTGATGACCACCTCGCAG   
3             20      60.04  60.000000   CCCTATATCTGTGGCTGCCG   

   pRight_EndStability  Penalty  Product_size         Amplicon_type Redesign  \
0                 4.26     0.29           879  Gene_specific:Rv0678        -   
1                 4.18     1.24           924          Non_specific        -   
2                 5.18     0.21           793          Non_specific        -   
3                 5.69     0.28           810          Non_specific        -   

      Designed_ranges  
0    [778990, 779790]  
1    [760663, 761463]  
2  [2155102, 2155802]  
3  [2288565, 2289265]

In [148]:
def len_calc(l):
    coords = [int(x.replace('[', '').replace(']', '')) for x in l.split(',')]
    return coords[1] - coords[0]

accepted_primer['Desgined_size'] =  list(map(len_calc, accepted_primer['Designed_ranges']))

In [150]:
print(len_calc(accepted_primer['Designed_ranges'][0]))

800


In [138]:
import pandas as pd
import re

# Sample DataFrame
data = {
    'ID': ['A1-sp', 'A1-ns', 'A1-ns', 'A2-ns'],
    'ID2': ['P1-sp-L1', 'P1-ns-L1', 'P1-ns-L1','P2-ns-L1'],
    # 'ID3': ['P1-sp-R1', 'P1-sp-R1', 'P1-ns-R1', 'P1-ns-R2'],
    # Other columns...
}
df = pd.DataFrame(data)

print(df)
print('==================')

def increment_id_with_regex(df, col_name):
    counter = {}  # Track counts of each ID
    new_ids = []  # Store updated IDs

    for id in df[col_name]:
        # original_id = id
        # Use regex to find all numbers in the ID
        if 'User' in id:
            continue
        numbers = re.findall(r'\d+', id)

        if numbers:
            first_number = numbers[0]  # Focus on the first number found
            if len(id.split('-')) > 2:
                check_id = '-'.join(id.split('-')[:-1])
            else:
                check_id = id
            new_id = id
            while check_id in counter:                
                # Increment the first number found
                new_number = str(int(first_number) + 1)
                # Replace the first occurrence of the number with its incremented value
                new_id = re.sub(r'(?<!\d)'+first_number+r'(?!\d)', new_number, new_id, count=1)
                check_id = re.sub(r'(?<!\d)'+first_number+r'(?!\d)', new_number, check_id, count=1)
                print(check_id)
                print(new_id)
                first_number = new_number  # Update first_number for potential next iteration
                
            counter[check_id] = True  # Mark this new ID as seen
            new_ids.append(new_id)  # Add to the list of new IDs
        # else:
        #     # If no number is found, just append the ID as is
        #     new_ids.append(id)

        # Update counter for the original ID if it's not already there to handle non-duplicates correctly
        # if original_id not in counter:
        #     counter[original_id] = True

    df[col_name] = new_ids  # Update the column with new IDs

# Apply the function to the columns of interest
# increment_id_with_regex(df, 'ID')
increment_id_with_regex(df, 'ID2')
    # increment_id_with_regex(df, 'ID3')

print(df)


      ID       ID2
0  A1-sp  P1-sp-L1
1  A1-ns  P1-ns-L1
2  A1-ns  P1-ns-L1
3  A2-ns  P2-ns-L1
P2-ns
P2-ns-L1
P3-ns
P3-ns-L1
      ID       ID2
0  A1-sp  P1-sp-L1
1  A1-ns  P1-ns-L1
2  A1-ns  P2-ns-L1
3  A2-ns  P3-ns-L1


In [ ]:
# def increment_id_with_regex(df, col_name):
        #     counter = {}  # Track counts of each ID
        #     new_ids = []  # Store updated IDs

        #     for id in df[col_name]:
        #         original_id = id
        #         # Use regex to find all numbers in the ID
        #         numbers = re.findall(r'\d+', id)
        #         if numbers:
        #             last_number = numbers[-1]  # Focus on the last number found
        #             new_id = id
        #             while new_id in counter:
        #                 # Increment the last number found
        #                 new_number = str(int(last_number) + 1)
        #                 # Replace the last occurrence of the number with its incremented value
        #                 new_id = re.sub(r'(?<!\d)'+last_number+r'(?!\d)', new_number, new_id, count=1)
        #                 last_number = new_number  # Update last_number for potential next iteration

        #             counter[new_id] = True  # Mark this new ID as seen
        #             new_ids.append(new_id)  # Add to the list of new IDs
        #         else:
        #             # If no number is found, just append the ID as is
        #             new_ids.append(id)

        #         # Update counter for the original ID if it's not already there to handle non-duplicates correctly
        #         if original_id not in counter:
        #             counter[original_id] = True

        #     df[col_name] = new_ids  # Update the column with new IDs

In [98]:
import pandas as pd

# Sample DataFrame
data = {
    'ID': ['A1-sp', 'A1-ns', 'A1-ns'],
    'ID2': ['P1-sp-L1', 'P1-ns-L1', 'P1-ns-L1'],
    # Other columns...
}
df = pd.DataFrame(data)

def increment_id_for_duplicates(df, col_name):
    counter = {}  # Track counts of each ID
    new_ids = []  # Store updated IDs
    
    for id in df[col_name]:
        prefix, suffix = id.rsplit('-', 1)  # Split ID into prefix and suffix
        print(prefix, suffix)
        base, num = prefix[:-2], int(prefix[-1])  # Assume last character is a number
        
        # Generate new ID with incremented number if duplicate
        while id in counter:
            num += 1
            id = f"{base}{num}-{suffix}"
        
        counter[id] = True  # Mark this ID as seen
        new_ids.append(id)  # Add to the list of new IDs
    
    df[col_name] = new_ids  # Update the column with new IDs

# Apply the function to the columns of interest
increment_id_for_duplicates(df, 'ID')
increment_id_for_duplicates(df, 'ID2')

print(df)


A1 sp
A1 ns
A1 ns
P1-sp L1


ValueError: invalid literal for int() with base 10: 'p'

In [72]:
v = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [74]:
v[v['gene']=='rpoB']

sample_id  genome_pos  gene       change      freq  \
0       SRR20969411      761155  rpoB  p.Ser450Leu  0.996825   
4       ERR10748278      761155  rpoB  p.Ser450Leu  1.000000   
8       ERR11081049      761155  rpoB  p.Ser450Leu  1.000000   
14      ERR11043928      761155  rpoB  p.Ser450Leu  1.000000   
25       ERR2864258      761139  rpoB  p.His445Asp  1.000000   
...             ...         ...   ...          ...       ...   
266444   SRR1165253      761155  rpoB  p.Ser450Leu  1.000000   
266449  SRR18715169      761109  rpoB  p.Asp435Tyr  1.000000   
266452   ERR2510245      761155  rpoB  p.Ser450Leu  1.000000   
266457   ERR4797048      761155  rpoB  p.Ser450Leu  1.000000   
266465   ERR7911528      761155  rpoB  p.Ser450Leu  1.000000   

                    type            sublin      drtype       drugs  
0       missense_variant      lineage2.2.1      MDR-TB  rifampicin  
4       missense_variant      lineage2.2.1      MDR-TB  rifampicin  
8       missense_variant      lineage4.6.5      MDR-TB  rifampicin  
14      missense_variant      lineage4.3.3  Pre-XDR-TB  rifampicin  
25      missense_variant    lineage4.1.1.3      MDR-TB  rifampicin  
...                  ...               ...         ...         ...  
266444  missense_variant  lineage4.4.1.1.1      MDR-TB  rifampicin  
266449  missense_variant        lineage4.8      MDR-TB  rifampicin  
266452  missense_variant          lineage3      MDR-TB  rifampicin  
266457  missense_variant      lineage2.2.1  Pre-XDR-TB  rifampicin  
266465  missense_variant      lineage2.2.2  Pre-XDR-TB  rifampicin  

[43364 rows x 9 columns]

In [94]:
def pop_first_item(d):
    if d:
        # Get the first key in the dictionary
        first_key = next(iter(d))
        # Remove and return the key-value pair
        return first_key, d.pop(first_key)
    else:
        raise KeyError('pop_first_item(): dictionary is empty')

# Example usage
d = {'a': 1, 'b': 2, 'c': 3}
key, value = pop_first_item(d)
print(f'Removed item: {key}, {value}')
print(f'Remaining dictionary: {d}')


Removed item: a, 1
Remaining dictionary: {'b': 2, 'c': 3}


In [95]:
d

{'b': 2, 'c': 3}

In [70]:
segmented = '400,1000,50,2'
segmented = segmented.strip().split(',')
segmented = [int(i.strip()) for i in segmented]
amplicon_sizes = np.arange(segmented[0], segmented[1]+1, segmented[2]).tolist()*2
amplicon_sizes.sort(reverse=True)
value_counts = {value: amplicon_sizes.count(value) for value in set(amplicon_sizes)}
value_counts = dict(sorted(value_counts.items(), reverse=True))


print(value_counts)
print(amplicon_sizes)
print(value_counts)

{1000: 2, 950: 2, 900: 2, 850: 2, 800: 2, 750: 2, 700: 2, 650: 2, 600: 2, 550: 2, 500: 2, 450: 2, 400: 2}
[1000, 1000, 950, 950, 900, 900, 850, 850, 800, 800, 750, 750, 700, 700, 650, 650, 600, 600, 550, 550, 500, 500, 450, 450, 400, 400]
{1000: 2, 950: 2, 900: 2, 850: 2, 800: 2, 750: 2, 700: 2, 650: 2, 600: 2, 550: 2, 500: 2, 450: 2, 400: 2}


In [76]:
v

sample_id  genome_pos  gene          change      freq  \
0       SRR20969411      761155  rpoB     p.Ser450Leu  0.996825   
1       SRR20969411      781687  rpsL      p.Lys43Arg  1.000000   
2       SRR20969411     2155168  katG     p.Ser315Thr  1.000000   
3       SRR20969411     4247729  embB     p.Gly406Ser  0.997268   
4       ERR10748278      761155  rpoB     p.Ser450Leu  1.000000   
...             ...         ...   ...             ...       ...   
266471   ERR7911528     3087715   ald  c.897_898dupGA  0.459770   
266472   ERR7911528     4247431  embB     p.Met306Ile  1.000000   
266473   ERR7911528     4326333  ethA     p.Ala381Pro  1.000000   
266474   ERR7911528     4407967   gid      p.Leu79Ser  1.000000   
266475    ERR718460     1472359   rrs        n.514A>C  1.000000   

                                      type        sublin      drtype  \
0                         missense_variant  lineage2.2.1      MDR-TB   
1                         missense_variant  lineage2.2.1      MDR-TB   
2                         missense_variant  lineage2.2.1      MDR-TB   
3                         missense_variant  lineage2.2.1      MDR-TB   
4                         missense_variant  lineage2.2.1      MDR-TB   
...                                    ...           ...         ...   
266471                  frameshift_variant  lineage2.2.2  Pre-XDR-TB   
266472                    missense_variant  lineage2.2.2  Pre-XDR-TB   
266473                    missense_variant  lineage2.2.2  Pre-XDR-TB   
266474                    missense_variant  lineage2.2.2  Pre-XDR-TB   
266475  non_coding_transcript_exon_variant  lineage2.2.1       Other   

               drugs  
0         rifampicin  
1       streptomycin  
2          isoniazid  
3         ethambutol  
4         rifampicin  
...              ...  
266471   cycloserine  
266472    ethambutol  
266473   ethionamide  
266474  streptomycin  
266475  streptomycin  

[266476 rows x 9 columns]

In [92]:
def optimal_amplicon_coverage_with_quantities(df, amplicon_quantities):
    """
    Finds the optimal combination of amplicons of various lengths and quantities to cover given genomic positions.
    
    Parameters:
    - df: DataFrame with a 'genomic_pos' column representing genomic positions.
    - amplicon_quantities: A dictionary with amplicon lengths as keys and the quantities available as values.
    
    Returns:
    - The total number of amplicons used and the chosen lengths for each segment along with their quantities.
    """
    # Ensure the genomic positions are sorted
    sorted_positions = df['genome_pos'].sort_values().unique()
    
    # Initialize counters and storage for results
    total_amplicons_used = 0
    coverage_plan = []  # To store the start position, length, and quantity used for each chosen amplicon
    
    current_position = sorted_positions[0]
    end_position = sorted_positions[-1]
    
    while current_position <= end_position:
        # Iterate through available amplicons, sorted by length descending to try the longest first
        print(current_position, end_position)
        for length, quantity in sorted(amplicon_quantities.items(), key=lambda x: -x[0]):
            print(length, quantity)
            if quantity > 0 and current_position + length - 1 <= end_position:
                # Amplicon can be used, update counters and results
                total_amplicons_used += 1
                coverage_plan.append((current_position, length))
                current_position += length  # Move to the next segment
                amplicon_quantities[length] -= 1  # Decrease the quantity of the used amplicon
                print('found')
                break
        else:
            # If no amplicon fits, break the loop and raise an error
            raise ValueError("Cannot cover all positions with available amplicon lengths and quantities.")
    
    return total_amplicons_used, coverage_plan


In [93]:
print(optimal_amplicon_coverage_with_quantities(v[v['gene']=='rpoB'], value_counts))

760314 762748
1000 0
950 2
found
761264 762748
1000 0
950 1
found
762214 762748
1000 0
950 0
900 2
850 2
800 2
750 2
700 2
650 2
600 2
550 2
500 2
found
762714 762748
1000 0
950 0
900 2
850 2
800 2
750 2
700 2
650 2
600 2
550 2
500 1
450 2
400 1


ValueError: Cannot cover all positions with available amplicon lengths and quantities.

In [50]:
v[v['gene']!= 'rpoC'].to_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv', index=False)

In [28]:
np.arange(400, 801, 50)

array([400, 450, 500, 550, 600, 650, 700, 750, 800])

In [31]:
int(20/len(np.arange(400, 801, 50)))

2

In [33]:
int(30/len(np.arange(400, 801, 50)))

3

In [39]:
np.sum([3,3,2,2,2,2,2,2,2])

20

In [43]:
import plotly.graph_objects as go

# Sample data for a bar chart
data = {'Category': [str(x) for x in np.arange(400, 801, 50)],
        'Values': [2,2,2,2,2,2,2,3,3]}

# Create a bar chart
fig = go.Figure([go.Bar(x=data['Category'], y=data['Values'])])

# Update layout for a better visual appearance
fig.update_layout(title_text='20 amplicons of 400-800bps (+50bps steps)',
                  xaxis_title='Amplicon size (bps)',
                  yaxis_title='Number of amplicons',
                  plot_bgcolor='white',
                  margin=dict(l=20, r=20, t=40, b=20),
                  width = 600)

# Show plot
fig.show()


In [44]:
import plotly.graph_objects as go

# Sample data for a bar chart
data = {'Category': [str(x) for x in np.arange(400, 801, 50)],
        'Values': [2,2,3,3,3,3,3,3,3]}

# Create a bar chart
fig = go.Figure([go.Bar(x=data['Category'], y=data['Values'])])

# Update layout for a better visual appearance
fig.update_layout(title_text='25 amplicons of 400-800bps (+50bps steps)',
                  xaxis_title='Amplicon size (bps)',
                  yaxis_title='Number of amplicons',
                  plot_bgcolor='white',
                  margin=dict(l=20, r=20, t=40, b=20),
                  width = 600)

# Show plot

In [96]:
import plotly.graph_objects as go

# Sample data for a bar chart
data = {'Category': [str(x) for x in np.arange(400, 801, 50)],
        'Values': [3,3,3,3,3,3,4,4,4]}

# Create a bar chart
fig = go.Figure([go.Bar(x=data['Category'], y=data['Values'])])

# Update layout for a better visual appearance
fig.update_layout(title_text='30 amplicons of 400-800bps (+50bps steps)',
                  xaxis_title='Amplicon size (bps)',
                  yaxis_title='Number of amplicons',
                  plot_bgcolor='white',
                  margin=dict(l=20, r=20, t=40, b=20),
                  width = 600)

# Show plot

In [3]:
gff3_df = pd.read_csv(gff3_file_path, sep='\t', comment='#', names=gff3_columns)

NameError: name 'gff3_file_path' is not defined

In [5]:
for x in genes_df['gene_name'].unique():
    print(x)

NameError: name 'genes_df' is not defined

In [3]:
import pandas as pd

# Define the path to your GFF3 file
def extract_gff3(gff3_file_path):
    # Read the GFF3 file
    gff3_columns = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
    gff3_df = pd.read_csv(gff3_file_path, sep='\t', comment='#', names=gff3_columns)

    # Filter to keep only gene entries and explicitly create a copy
    genes_df = gff3_df[gff3_df['type'] == 'gene'].copy()

    # Extract gene names from the attributes column
    genes_df['gene_id'] = genes_df['attributes'].apply(lambda x: x.split(';')[0].split('=')[1].split(':')[1])
    genes_df['gene_name'] = genes_df['attributes'].apply(lambda x: x.split(';')[1].split('=')[1])

    # Create a new DataFrame with the required information
    genes_info_df = genes_df[['gene_id', 'gene_name', 'seqid', 'start', 'end']]

    # Rename columns for clarity
    genes_info_df.columns = ['gene_id', 'gene_name', 'chr', 'start', 'end']
    return genes_info_df

genes_info_df = extract_gff3('/mnt/storage10/lwang/Projects/TOAST/toast/db/MTB-h37rv_asm19595v2-eg18.gff')

# Optionally, save this DataFrame to a CSV file
# output_csv_path = 'genes_info.csv'
# genes_info_df.to_csv(output_csv_path, index=False)

In [25]:
import pandas as pd

# Sample DataFrame
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'City': ['New York', 'Paris', 'Berlin', 'Paris', 'London'],
    'Age': [25, 30, 35, 40, 45]
}
df = pd.DataFrame(data)

# Define the values to filter by
cities_to_filter = ['Paris', 'London']
ages_to_filter = [30, 45]
all_filter =  ['Paris', 'London', 30, 45]

# Apply the .isin() method on each column separately and combine conditions
filtered_df = df[df['City'].isin(all_filter) | df['Age'].isin(all_filter)]

print(filtered_df)


    Name    City  Age
1    Bob   Paris   30
3  David   Paris   40
4    Eve  London   45


In [6]:
test_l = ['mmpr5']

In [7]:
t = genes_info_df[genes_info_df['gene_id'].isin(test_l) | genes_info_df['gene_name'].isin(test_l)]#[['start','end']]
print(t)

Empty DataFrame
Columns: [start, end]
Index: []


In [ ]:
with np.printoptions(threshold=np.inf):
    print(genes_info_df['gene_id'].unique())

In [21]:
'Rv0678' in genes_info_df['gene_id'].tolist()

True

In [26]:
np.round(59.97109789950406,3)

59.971

In [23]:
genes_info_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


gene_id       gene_name         chr    start      end
0       rv0001            dnaa  chromosome        1     1524
5       rv0002            dnan  chromosome     2052     3260
9       rv0003            recf  chromosome     3280     4437
13      rv0004  protein_coding  chromosome     4434     4997
17      rv0005            gyrb  chromosome     5240     7267
...        ...             ...         ...      ...      ...
16465  rv3920c  protein_coding  chromosome  4408334  4408897
16469  rv3921c  protein_coding  chromosome  4408969  4410069
16473  rv3922c  protein_coding  chromosome  4410053  4410415
16477  rv3923c            rnpa  chromosome  4410412  4410789
16481  rv3924c            rpmh  chromosome  4410786  4410929

[4018 rows x 5 columns]

In [17]:
print(genes_info_df[genes_info_df['gene_id']=='Rv0678'])

     gene_id       gene_name         chr   start     end
2837  Rv0678  protein_coding  Chromosome  778990  779487


In [13]:
with np.printoptions(threshold=np.inf):
    print(genes_info_df['gene_name'].unique())

['dnaA' 'dnaN' 'recF' 'protein_coding' 'gyrB' 'gyrA' 'ppiA' 'trpG' 'pknB'
 'pknA' 'pbpA' 'rodA' 'pstP' 'fhaB' 'fhaA' 'whiB5' 'bioF2' 'acpA' 'fadD34'
 'mtc28' 'leuS' 'ino1' 'ponA1' 'rpsF' 'ssb' 'rpsR1' 'rplI' 'dnaB' 'celA1'
 'vapB1' 'vapC1' 'icd2' 'sdaA' 'glyA2' 'hycD' 'hycP' 'hycQ' 'hycE' 'mtn'
 'ctpA' 'PPE1' 'fcoT' 'fadD10' 'nrp' 'ctpB' 'rpmB1' 'ctpI' 'PE_PGRS1'
 'gca' 'gmhA' 'gmhB' 'hddA' 'ldtA' 'oxyS' 'oxcA' 'fadD7' 'fusA2'
 'PE_PGRS2' 'pepA' 'treS' 'mak' 'fbpC' 'htdZ' 'fadE1' 'fgd2' 'ephF'
 'cyp138' 'msrA' 'PE1' 'PE2' 'ptbB' 'fadE2' 'pntAa' 'pntAb' 'pntB' 'PE3'
 'PE4' 'adhE1' 'TB18.5' 'mce1R' 'fadD5' 'yrbE1A' 'yrbE1B' 'mce1A' 'mce1B'
 'mce1C' 'mce1D' 'lprK' 'mce1F' 'lprO' 'sigG' 'bglS' 'mymT' 'ilvD' 'zmp1'
 'mmpL11' 'mmpL3' 'pckA' 'nadR' 'fadD4' 'fadE3' 'lipW' 'lipC' 'echA1'
 'php' 'fadE4' 'nrdB' 'gabD1' 'aftD' 'lpqI' 'vapB24' 'vapC24' 'htdX'
 'fabG4' 'fadA2' 'fadE5' 'hsp' 'nirB' 'nirD' 'cobU' 'cobQ1' 'PPE2' 'narK3'
 'aac' 'oplA' 'narU' 'fadD2' 'fadE6' 'vapC25' 'PE_PGRS3' 'PE_PGRS4

In [31]:
test_df = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [32]:
test_df

sample_id  genome_pos  gene          change      freq  \
0       SRR20969411      761155  rpoB     p.Ser450Leu  0.996825   
1       SRR20969411      781687  rpsL      p.Lys43Arg  1.000000   
2       SRR20969411     2155168  katG     p.Ser315Thr  1.000000   
3       SRR20969411     4247729  embB     p.Gly406Ser  0.997268   
4       ERR10748278      761155  rpoB     p.Ser450Leu  1.000000   
...             ...         ...   ...             ...       ...   
268363   ERR7911528     3087715   ald  c.897_898dupGA  0.459770   
268364   ERR7911528     4247431  embB     p.Met306Ile  1.000000   
268365   ERR7911528     4326333  ethA     p.Ala381Pro  1.000000   
268366   ERR7911528     4407967   gid      p.Leu79Ser  1.000000   
268367    ERR718460     1472359   rrs        n.514A>C  1.000000   

                                      type        sublin      drtype  \
0                         missense_variant  lineage2.2.1      MDR-TB   
1                         missense_variant  lineage2.2.1      MDR-TB   
2                         missense_variant  lineage2.2.1      MDR-TB   
3                         missense_variant  lineage2.2.1      MDR-TB   
4                         missense_variant  lineage2.2.1      MDR-TB   
...                                    ...           ...         ...   
268363                  frameshift_variant  lineage2.2.2  Pre-XDR-TB   
268364                    missense_variant  lineage2.2.2  Pre-XDR-TB   
268365                    missense_variant  lineage2.2.2  Pre-XDR-TB   
268366                    missense_variant  lineage2.2.2  Pre-XDR-TB   
268367  non_coding_transcript_exon_variant  lineage2.2.1       Other   

               drugs  
0         rifampicin  
1       streptomycin  
2          isoniazid  
3         ethambutol  
4         rifampicin  
...              ...  
268363   cycloserine  
268364    ethambutol  
268365   ethionamide  
268366  streptomycin  
268367  streptomycin  

[268368 rows x 9 columns]

In [36]:
import pandas as pd

# Define your set of ranges
# Example: [(1, 5), (10, 12)] represents two ranges: from 1 to 5 and from 10 to 12
ranges = [(1, 5000), (10000, 120000)]

# Initialize an empty list to store all genomic positions
all_positions = []

# Iterate through each range and generate positions
for i, row in t.iterrows():
    # Generate positions for the current range and append to the list
    all_positions.extend(range(row.start, row.end + 1))  # end + 1 because range is exclusive at the end

# Create a DataFrame
df = pd.DataFrame({
    'genome_pos': all_positions,
    'weight': 1
})

print(df)


      genomic_pos  weights
0               1        1
1               2        1
2               3        1
3               4        1
4               5        1
...           ...      ...
3886         4433        1
3887         4434        1
3888         4435        1
3889         4436        1
3890         4437        1

[3891 rows x 2 columns]


In [107]:
from Bio import SeqIO
import pandas as pd

# Path to your GFF file
gff_file = '/mnt/storage10/lwang/Projects/TOAST/toast/db/MTB-h37rv_asm19595v2-eg18.gff'

# Initialize a list to store the extracted information
gene_info = []

# Parse the GFF file
for record in SeqIO.parse(gff_file, "genbank"):
    print(1)
    print(record)
    for feature in record.features:
        print(1)
        print(feature)
        if feature.type == "gene":
            # Extract gene name, some GFF files might use 'ID' or other attributes for the gene name
            gene_name = feature.qualifiers.get('Name', [''])[0] or feature.qualifiers.get('ID', [''])[0]
            # Extract chromosome name
            chromosome = record.id
            # Extract start and end positions of the gene
            start = feature.location.start
            end = feature.location.end
            # Append the information to the list
            gene_info.append([gene_name, chromosome, start, end])

# Create a DataFrame
df_genes = pd.DataFrame(gene_info, columns=['Gene Name', 'Chromosome', 'Start', 'End'])

# Display the DataFrame
print(df_genes)

# Optionally, save the DataFrame to a CSV file
# df_genes.to_csv('genes_ranges.csv', index=False)


1
Empty DataFrame
Columns: [Gene Name, Chromosome, Start, End]
Index: []


In [100]:
df_genes

Empty DataFrame
Columns: [Gene Name, Chromosome, Start, End]
Index: []

In [91]:
pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/test_df.csv').iloc[0]
pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/test_df copy.csv', header=None)

0                     1        2
0  CACTCCGAAGAAGCCGAACT  GTGACCAACTGCGATCCGTA  Primer1

In [96]:
accepted_primers = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/output/Amplicon_design_output/Primer_design-accepted_primers-40-400.csv')
full_data = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [98]:
full_data['gene'].unique()

array(['rpoB', 'rpsL', 'katG', 'embB', 'fabG1', 'ethR', 'pncA', 'tlyA',
       'gyrA', 'rrs', 'mmpR5', 'gid', 'eis', 'ethA', 'folC', 'rpoC',
       'embA', 'alr', 'gyrB', 'inhA', 'ahpC', 'ald', 'thyX', 'thyA',
       'rplC', 'ddn', 'embC', 'fbiA', 'kasA', 'rrl', 'embR', 'panD',
       'ribD', 'rpsA', 'fgd1'], dtype=object)

In [39]:
dtypes = {}

for column in full_data.columns:
    # Get the data type of the column
    dtype = str(full_data[column].dtype)
    # Add the column and its data type to the dictionary
    dtypes[column] = dtype
amp_snp = pd.DataFrame(columns=dtypes.keys()).astype(dtypes)

In [48]:
amp_snp

sample_id  genome_pos  gene       change      freq  \
0       SRR20969411      761155  rpoB  p.Ser450Leu  0.996825   
4       ERR10748278      761155  rpoB  p.Ser450Leu  1.000000   
8       ERR11081049      761155  rpoB  p.Ser450Leu  1.000000   
14      ERR11043928      761155  rpoB  p.Ser450Leu  1.000000   
25       ERR2864258      761139  rpoB  p.His445Asp  1.000000   
...             ...         ...   ...          ...       ...   
268336   SRR1165253      761155  rpoB  p.Ser450Leu  1.000000   
268341  SRR18715169      761109  rpoB  p.Asp435Tyr  1.000000   
268344   ERR2510245      761155  rpoB  p.Ser450Leu  1.000000   
268349   ERR4797048      761155  rpoB  p.Ser450Leu  1.000000   
268357   ERR7911528      761155  rpoB  p.Ser450Leu  1.000000   

                    type            sublin      drtype       drugs  
0       missense_variant      lineage2.2.1      MDR-TB  rifampicin  
4       missense_variant      lineage2.2.1      MDR-TB  rifampicin  
8       missense_variant      lineage4.6.5      MDR-TB  rifampicin  
14      missense_variant      lineage4.3.3  Pre-XDR-TB  rifampicin  
25      missense_variant    lineage4.1.1.3      MDR-TB  rifampicin  
...                  ...               ...         ...         ...  
268336  missense_variant  lineage4.4.1.1.1      MDR-TB  rifampicin  
268341  missense_variant        lineage4.8      MDR-TB  rifampicin  
268344  missense_variant          lineage3      MDR-TB  rifampicin  
268349  missense_variant      lineage2.2.1  Pre-XDR-TB  rifampicin  
268357  missense_variant      lineage2.2.2  Pre-XDR-TB  rifampicin  

[42357 rows x 9 columns]

In [84]:
full_data[full_data['gene']=='fabG1']

sample_id  genome_pos   gene    change     freq  \
5       ERR10748278     1673425  fabG1  c.-15C>T  1.00000   
17      ERR11043928     1673425  fabG1  c.-15C>T  1.00000   
41       ERR3287611     1673425  fabG1  c.-15C>T  0.97561   
42       ERR4828904     1673425  fabG1  c.-15C>T  1.00000   
58       ERR4972558     1673425  fabG1  c.-15C>T  1.00000   
...             ...         ...    ...       ...      ...   
268284  SRR18715029     1673425  fabG1  c.-15C>T  1.00000   
268308   ERR4797355     1673425  fabG1  c.-15C>T  1.00000   
268312   ERR4818250     1673425  fabG1  c.-15C>T  1.00000   
268316  SRR21658376     1673425  fabG1  c.-15C>T  1.00000   
268360   ERR7911528     1673425  fabG1  c.-15C>T  1.00000   

                         type          sublin      drtype  \
5       upstream_gene_variant    lineage2.2.1      MDR-TB   
17      upstream_gene_variant    lineage4.3.3  Pre-XDR-TB   
41      upstream_gene_variant    lineage4.3.3      MDR-TB   
42      upstream_gene_variant  lineage4.1.2.1       HR-TB   
58      upstream_gene_variant    lineage4.3.3      MDR-TB   
...                       ...             ...         ...   
268284  upstream_gene_variant    lineage2.2.1      MDR-TB   
268308  upstream_gene_variant    lineage2.2.1      MDR-TB   
268312  upstream_gene_variant        lineage3       HR-TB   
268316  upstream_gene_variant    lineage2.2.1       HR-TB   
268360  upstream_gene_variant    lineage2.2.2  Pre-XDR-TB   

                        drugs  
5       isoniazid,ethionamide  
17      isoniazid,ethionamide  
41      isoniazid,ethionamide  
42      isoniazid,ethionamide  
58      isoniazid,ethionamide  
...                       ...  
268284  isoniazid,ethionamide  
268308  isoniazid,ethionamide  
268312  isoniazid,ethionamide  
268316  isoniazid,ethionamide  
268360  isoniazid,ethionamide  

[14208 rows x 9 columns]

In [83]:
full_data[full_data['gene']=='inhA']

sample_id  genome_pos  gene       change      freq  \
215      ERR1815551     1674481  inhA   p.Ser94Ala  1.000000   
259     ERR11081269     1674262  inhA   p.Ile21Val  1.000000   
307      SRR1166935     1674481  inhA   p.Ser94Ala  1.000000   
386      ERR7362009     1674782  inhA  p.Ile194Thr  1.000000   
511      ERR7737841     1674048  inhA    c.-154G>A  1.000000   
...             ...         ...   ...          ...       ...   
267884   ERR4676889     1674481  inhA   p.Ser94Ala  1.000000   
267889   ERR1034814     1674481  inhA   p.Ser94Ala  0.989247   
267942   SRR1049045     1674048  inhA    c.-154G>A  1.000000   
268007   ERR2184615     1674048  inhA    c.-154G>A  1.000000   
268095   SRR1019138     1674048  inhA    c.-154G>A  1.000000   

                         type          sublin      drtype  \
215          missense_variant  lineage4.3.4.2  Pre-XDR-TB   
259          missense_variant      lineage4.7      MDR-TB   
307          missense_variant  lineage4.2.2.2  Pre-XDR-TB   
386          missense_variant    lineage2.2.1      MDR-TB   
511     upstream_gene_variant    lineage2.2.1      MDR-TB   
...                       ...             ...         ...   
267884       missense_variant  lineage4.3.4.2      MDR-TB   
267889       missense_variant  lineage4.3.4.2  Pre-XDR-TB   
267942  upstream_gene_variant    lineage2.2.2  Pre-XDR-TB   
268007  upstream_gene_variant  lineage4.1.2.1       HR-TB   
268095  upstream_gene_variant    lineage1.2.2      MDR-TB   

                        drugs  
215     isoniazid,ethionamide  
259                 isoniazid  
307     isoniazid,ethionamide  
386     isoniazid,ethionamide  
511     isoniazid,ethionamide  
...                       ...  
267884  isoniazid,ethionamide  
267889  isoniazid,ethionamide  
267942  isoniazid,ethionamide  
268007  isoniazid,ethionamide  
268095  isoniazid,ethionamide  

[4168 rows x 9 columns]

In [53]:
accepted_primers

index   pLeft_ID  pLeft_coord  pLeft_length   pLeft_Tm   pLeft_GC  \
0       0   P1-ns-L1       761067            20  59.965104  55.000000   
1       0   P2-ns-L1      2155074            20  59.827648  60.000000   
2       0   P3-ns-L1      4247492            20  60.104934  55.000000   
3       0   P4-ns-L1      2288903            20  60.179194  60.000000   
4       0   P5-ns-L1       781678            20  60.038159  55.000000   
5       1   P6-ns-L2      4247250            20  60.750911  60.000000   
6       0   P7-ns-L1      1673477            20  59.897519  60.000000   
7       0   P8-ns-L1         7344            20  60.109458  55.000000   
8       0   P9-ns-L1      2288651            20  59.203688  55.000000   
9       0  P10-ns-L1      1472411            20  60.038526  55.000000   
10      0  P11-ns-L1      4407894            20  60.386858  55.000000   
11      0  P12-ns-L1      1473053            20  60.035152  55.000000   
12      0  P13-ns-L1      2715439            20  60.179137  50.000000   
13      0  P14-ns-L1      1472086            20  60.540365  55.000000   
14      0  P15-ns-L1      4326166            20  60.034089  60.000000   
15      0  P16-ns-L1      4243328            20  59.689146  55.000000   
16      0  P17-ns-L1      4327146            20  60.107508  55.000000   
17      0  P18-ns-L1      4247819            19  60.004320  52.631579   
18      0  P19-ns-L1      4407613            20  59.901272  50.000000   
19      0  P20-ns-L1      4327531            20  60.460284  60.000000   
20      0  P21-ns-L1      4326665            20  60.463474  60.000000   
21      0  P22-ns-L1      1674181            20  59.972414  55.000000   
22      0  P23-ns-L1      2726206            20  59.688210  55.000000   
23      0  P24-ns-L1       764814            20  60.036952  55.000000   
24      0  P25-ns-L1      2747213            20  61.517210  65.000000   
25      0  P26-ns-L1         6659            20  59.825155  55.000000   
26      0  P27-ns-L1      2288978            20  60.108615  60.000000   
27      0  P28-ns-L1      4249439            20  60.530650  55.000000   
28      0  P29-ns-L1      1674559            20  59.969765  55.000000   
29      0  P30-ns-L1      2155516            20  59.964542  60.000000   
30      0  P31-ns-L1      4325884            20  59.834056  55.000000   
31      0  P32-ns-L1      4326347            20  59.898241  55.000000   
32      0  P33-ns-L1      3841195            19  60.226211  63.157895   
33      0  P34-ns-L1      3068072            20  59.837706  55.000000   
34      0  P35-ns-L1      1918397            20  59.831652  55.000000   
35      0  P36-ns-L1      3073840            20  59.829661  55.000000   
36      0  P37-ns-L1       762211            19  59.783290  63.157895   
37      0  P38-ns-L1      2154721            20  59.605435  60.000000   
38      0  P39-ns-L1      4326974            20  60.250209  55.000000   
39      0  P40-ns-L1       764453            20  59.973715  55.000000   

         pLeft_Sequences  pLeft_EndStability  pRight_ID  pRight_coord  \
0   TCAAGGAGTTCTTCGGCACC                5.01   P1-ns-R1        761385   
1   CTCTTCGTCAGCTCCCACTC                3.51   P2-ns-R1       2155374   
2   AGGATCCCTTCGGCTGGTAT                2.73   P3-ns-R1       4247838   
3   CCGTTCTCGTCGACTCCTTC                3.46   P4-ns-R1       2289232   
4   CACTCCGAAGAAGCCGAACT                3.01   P5-ns-R1        782004   
5   CTCAATTGCCCAGCTCCTCC                4.30   P6-ns-R2       4247563   
6   CGTATCCCGTTCAGTCCTGG                4.45   P7-ns-R1       1673777   
7   TCGAACCGGTTGACATCGSG                4.04   P8-ns-R1          7690   
8   CAACAGTTCATCCCGGTTCG                3.95   P9-ns-R1       2288973   
9   AGCTCGTAGGTGGTTTGTCG                4.35  P10-ns-R1       1472768   
10  TCTCGAAGAAACTCGGTGCG                5.34  P11-ns-R1       4408204   
11  CCYGGGCTTCACARATGCTA                3.49  P12-ns-R1       1473442   
12  TGCGAGAATCAGCGACGAAT                3.34  P13-ns-R1      

In [ ]:


for x,y in zip(accepted_primers['pLeft_coord'].tolist(), accepted_primers['pRight_coord'].tolist()):
    print(x,y)
    condition = (full_data['genome_pos'] >= x) & (full_data['genome_pos'] <= y)
    filtered_data = full_data[condition]
    # print(filtered_data)
    # Drop columns that are completely empty or all NA from filtered_data
    filtered_data = filtered_data.dropna(axis=1, how='all')
    amp_snp = pd.concat([amp_snp, filtered_data])


In [54]:
condition = (full_data['genome_pos'] >= 1673477) & (full_data['genome_pos'] <= 1673777)
filtered_data = full_data[condition]

In [60]:
print(full_data[full_data['gene']=='fabG1']['genome_pos'].unique().min())
print(full_data[full_data['gene']=='fabG1']['genome_pos'].unique().max())

1673373
1673432


In [66]:
pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/regions_copy.bed', sep='\t', header=0)[['start', 'end', 'gene']].to_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/regions.bed', sep='\t', header=0, index=False)

In [74]:
tbdb = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/tbdb.bed', sep='\t', header=None)

In [76]:
tbdb = tbdb[[0,1,2,4]]

In [77]:
tbdb.columns = ['Chromosome', 'start', 'end', 'gene']

In [78]:
tbdb.to_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/regions.bed', sep='\t', header=0, index=False)

In [64]:
pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/regions copy.bed', sep='\t', header=0)[['start', 'end', 'gene']]

start      end   gene
0   2517694  2519364   kasA
1   4407527  4408333    gid
2   1833377  1834986   rpsA
3   4036730  4040936   clpC
4   4239662  4249809   embC
5   4239662  4249809   embB
6   4239662  4249809   embA
7   4266952  4269832   ubiA
8   1471575  1477012    rrs
9   4326003  4328198  ethAR
10  2725476  2726779   ahpC
11  1917754  1918745   tlyA
12  2153234  2156705   katG
13   781310   781933   rpsL
14   759608   767319   rpoB
15   759608   767319   rpoC
16  1672456  1675010  fabG1
17  1672456  1675010   inhA
18  2713782  2716313    eis
19     4996     9817   gyrA
20     4996     9817   gyrB
21  4043040  4045209   panD
22  2287882  2289598   pncA
23   575348   576790   mshA
24  1416181  1417347   embR
25  3878588  3879631   rpoA
26  2101651  2103042    ndh

In [55]:
filtered_data

Empty DataFrame
Columns: [sample_id, genome_pos, gene, change, freq, type, sublin, drtype, drugs]
Index: []

In [50]:
full_data_gene = full_data['gene'].value_counts()
amp_snp_aligned = amp_snp['gene'].value_counts().reindex(full_data_gene.index).fillna(0)

In [51]:
amp_snp_aligned

rpoB     84714.0
katG     39140.0
pncA     54882.0
embB     34183.0
rpsL     22971.0
rrs      17780.0
fabG1        0.0
gyrA     14048.0
ethA     12177.0
gid       8707.0
eis          0.0
inhA      2392.0
embA         0.0
rpoC      1536.0
ahpC        13.0
gyrB       914.0
folC       835.0
ethR      1425.0
thyX         0.0
tlyA       629.0
thyA        76.0
alr         66.0
ald          0.0
embC         0.0
mmpR5        0.0
rplC         0.0
ddn          0.0
panD         0.0
rrl          0.0
kasA         0.0
embR         0.0
fbiA         0.0
rpsA         0.0
ribD         0.0
fgd1         0.0
Name: gene, dtype: float64

In [42]:
amp_snp

Empty DataFrame
Columns: [sample_id, genome_pos, gene, change, freq, type, sublin, drtype, drugs]
Index: []

In [22]:
with open('/mnt/storage10/lwang/Projects/TOAST/toast/db/default_primer_design_setting.json', 'r') as file:
    primer_settings = json.load(file)

In [28]:
a = [1,2,3,4,]

In [34]:
a = ['=']*3 +a

In [35]:
a

['=', '=', '=', 1, 2, 3, 4, '=', '=', '=']

In [26]:
import json

# Load primer settings from a JSON file
with open('/mnt/storage10/lwang/Projects/TOAST/toast/db/default_primer_design_setting.json', 'r') as file:
    primer_settings = json.load(file)

# Replace the placeholder with the actual size range
size_range = '400-800'  # Replace with actual size range
primer_settings['PRIMER_PRODUCT_SIZE_RANGE'] = size_range

# Extract sequence from fasta
# genome = extract_sequence_from_fasta(0, genome_size(ref_genome), padding=0, fasta_file=ref_genome)
# ... rest of your code ...

# try:
#     results = bindings.design_primers(
#         seq_args={
#             'SEQUENCE_ID': 'Amplicon',
#             'SEQUENCE_TEMPLATE': sequence,
#             'SEQUENCE_INCLUDED_REGION': [padding, len(sequence) - padding],
#             'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': f'{ok_region_list[0]},{ok_region_list[1]},{ok_region_list[2]},{ok_region_list[3]}'
#         },
#         global_args=primer_settings  # Use the loaded primer settings
#     )
#     # ... rest of your code ...
# except:
#     # Handle exceptions


In [27]:
primer_settings

{'PRIMER_NUM_RETURN': 16,
 'PRIMER_PICK_INTERNAL_OLIGO': 0,
 'PRIMER_INTERNAL_MAX_SELF_END': 8,
 'PRIMER_MIN_SIZE': 18,
 'PRIMER_MAX_SIZE': 24,
 'PRIMER_MIN_TM': 58.0,
 'PRIMER_MAX_TM': 80.0,
 'PRIMER_MIN_GC': 45.0,
 'PRIMER_MAX_GC': 68.0,
 'PRIMER_MAX_POLY_X': 5,
 'PRIMER_INTERNAL_MAX_POLY_X': 5,
 'PRIMER_SALT_MONOVALENT': 50.0,
 'PRIMER_DNA_CONC': 50.0,
 'PRIMER_MAX_NS_ACCEPTED': 0,
 'PRIMER_MAX_SELF_ANY': 10,
 'PRIMER_MAX_SELF_END': 8,
 'PRIMER_PAIR_MAX_COMPL_ANY': 9,
 'PRIMER_PAIR_MAX_COMPL_END': 8,
 'PRIMER_PRODUCT_SIZE_RANGE': '400-800'}

In [2]:
v = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [36]:
v['gene'].unique()

array(['rpoB', 'rpsL', 'katG', 'embB', 'fabG1', 'ethR', 'pncA', 'tlyA',
       'gyrA', 'rrs', 'mmpR5', 'gid', 'eis', 'ethA', 'folC', 'rpoC',
       'embA', 'alr', 'gyrB', 'inhA', 'ahpC', 'ald', 'thyX', 'thyA',
       'rplC', 'ddn', 'embC', 'fbiA', 'kasA', 'rrl', 'embR', 'panD',
       'ribD', 'rpsA', 'fgd1'], dtype=object)

In [8]:
len(np.unique(v['drugs'].values))

24

In [9]:
np.unique(v['drugs'].values)

array(['aminoglycosides,amikacin', 'capreomycin',
       'capreomycin,capreomycin', 'clofazimine,bedaquiline',
       'cycloserine', 'delamanid', 'ethambutol', 'ethionamide',
       'ethionamide,ethionamide', 'isoniazid', 'isoniazid,ethionamide',
       'isoniazid,isoniazid', 'kanamycin', 'kanamycin,amikacin',
       'kanamycin,capreomycin,aminoglycosides,amikacin',
       'kanamycin,kanamycin', 'linezolid',
       'ofloxacin,moxifloxacin,levofloxacin,fluoroquinolones,ciprofloxacin',
       'para-aminosalicylic_acid', 'pyrazinamide',
       'pyrazinamide,pyrazinamide', 'rifampicin', 'rifampicin,rifampicin',
       'streptomycin'], dtype=object)

In [13]:
_ = []
for x in np.unique(v['drugs'].values):
    _.extend(x.split(','))

In [16]:
len(np.unique(_))

21

In [60]:
map_primer = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output/Amplicon_design_output/Amplicon_mapped-3-400.bed', header=None, sep='\t')
spol_primer = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/spoligo_primer.bed', header=None, sep='\t')

In [57]:
total_primer = pd.concat([map_primer, spol_primer])

In [58]:
total_primer

0        1        2                     3  4  5        6  \
0   Chromosome   781678   782004  Designed-A1-UserLeft  0  .   781678   
1   Chromosome   781678   782004           A1-UserLeft  0  .   781678   
2   Chromosome   781678   781698      Primer1-UserLeft  0  .   781678   
3   Chromosome   781984   782004     Primer1-UserRight  0  .   781984   
4   Chromosome   781585   781985        Designed-A2-sp  0  .   781585   
..         ...      ...      ...                   ... .. ..      ...   
75  Chromosome  3119555  3119573      Spol_Rprimer_DRb  0  -  3119555   
76  Chromosome  3119483  3119501      Spol_Rprimer_DRb  0  -  3119483   
77  Chromosome  3119410  3119428      Spol_Rprimer_DRb  0  -  3119410   
78  Chromosome  3119334  3119352      Spol_Rprimer_DRb  0  -  3119334   
79  Chromosome  3119184  3119202      Spol_Rprimer_DRb  0  -  3119184   

          7        8  
0    782004  0,0,255  
1    782004  0,255,0  
2    781698  255,0,0  
3    782004  128,0,0  
4    781985  0,0,255  
..      ...      ...  
75  3119573  128,0,0  
76  3119501  128,0,0  
77  3119428  128,0,0  
78  3119352  128,0,0  
79  3119202  128,0,0  

[92 rows x 9 columns]

In [61]:
total_primer.columns = ['col0', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8']

In [62]:
total_primer

col0     col1     col2                  col3  col4 col5     col6  \
0   Chromosome   781678   782004  Designed-A1-UserLeft     0    .   781678   
1   Chromosome   781678   782004           A1-UserLeft     0    .   781678   
2   Chromosome   781678   781698      Primer1-UserLeft     0    .   781678   
3   Chromosome   781984   782004     Primer1-UserRight     0    .   781984   
4   Chromosome   781585   781985        Designed-A2-sp     0    .   781585   
..         ...      ...      ...                   ...   ...  ...      ...   
75  Chromosome  3119555  3119573      Spol_Rprimer_DRb     0    -  3119555   
76  Chromosome  3119483  3119501      Spol_Rprimer_DRb     0    -  3119483   
77  Chromosome  3119410  3119428      Spol_Rprimer_DRb     0    -  3119410   
78  Chromosome  3119334  3119352      Spol_Rprimer_DRb     0    -  3119334   
79  Chromosome  3119184  3119202      Spol_Rprimer_DRb     0    -  3119184   

       col7     col8  
0    782004  0,0,255  
1    782004  0,255,0  
2    781698  255,0,0  
3    782004  128,0,0  
4    781985  0,0,255  
..      ...      ...  
75  3119573  128,0,0  
76  3119501  128,0,0  
77  3119428  128,0,0  
78  3119352  128,0,0  
79  3119202  128,0,0  

[92 rows x 9 columns]

In [51]:
pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output/Amplicon_design_output/Amplicon_mapped-2-400.bed', sep='\t', header=None)

0         1         2                  3    4    5         6   \
0   Chromosome  781678.0  782004.0     A1-UserDefined  0.0    .  781678.0   
1   Chromosome  781678.0  781698.0   Primer1-UserLeft  0.0    .  781678.0   
2   Chromosome  781984.0  782004.0  Primer1-UserRight  0.0    .  781984.0   
3   Chromosome  781585.0  781985.0     Designed-A1-sp  0.0    .  781585.0   
4   Chromosome  781678.0  782004.0              A1-sp  0.0    .  781678.0   
..         ...       ...       ...                ...  ...  ...       ...   
82         NaN       NaN       NaN                NaN  NaN  NaN       NaN   
83         NaN       NaN       NaN                NaN  NaN  NaN       NaN   
84         NaN       NaN       NaN                NaN  NaN  NaN       NaN   
85         NaN       NaN       NaN                NaN  NaN  NaN       NaN   
86         NaN       NaN       NaN                NaN  NaN  NaN       NaN   

          7        8           9          10         11                12  \
0   782004.0  0,255,0         NaN        NaN        NaN               NaN   
1   781698.0  255,0,0         NaN        NaN        NaN               NaN   
2   782004.0  128,0,0         NaN        NaN        NaN               NaN   
3   781985.0  0,0,255         NaN        NaN        NaN               NaN   
4   782004.0  0,255,0         NaN        NaN        NaN               NaN   
..       ...      ...         ...        ...        ...               ...   
82       NaN      NaN  Chromosome  3119555.0  3119573.0  Spol_Rprimer_DRb   
83       NaN      NaN  Chromosome  3119483.0  3119501.0  Spol_Rprimer_DRb   
84       NaN      NaN  Chromosome  3119410.0  3119428.0  Spol_Rprimer_DRb   
85       NaN      NaN  Chromosome  3119334.0  3119352.0  Spol_Rprimer_DRb   
86       NaN      NaN  Chromosome  3119184.0  3119202.0  Spol_Rprimer_DRb   

     13   14         15         16       17  
0   NaN  NaN        NaN        NaN      NaN  
1   NaN  NaN        NaN        NaN      NaN  
2   NaN  NaN        NaN        NaN      NaN  
3   NaN  NaN        NaN        NaN      NaN  
4   NaN  NaN        NaN        NaN      NaN  
..  ...  ...        ...        ...      ...  
82  0.0    -  3119555.0  3119573.0  128,0,0  
83  0.0    -  3119483.0  3119501.0  128,0,0  
84  0.0    -  3119410.0  3119428.0  128,0,0  
85  0.0    -  3119334.0  3119352.0  128,0,0  
86  0.0    -  3119184.0  3119202.0  128,0,0  

[87 rows x 18 columns]

In [6]:
# Sample data for the test DataFrame
data = {
    'Forward Primers': ['CACTCCGAAGAAGCCGAACT'],
    'Reverse Primers': ['TACGGATCGCAGTTGGTCAC'],
    'Name of Primer': ['Primer1']
}

# Create the DataFrame
test_df = pd.DataFrame(data)

test_df.head()  # Display the DataFrame


Forward Primers       Reverse Primers Name of Primer
0  CACTCCGAAGAAGCCGAACT  TACGGATCGCAGTTGGTCAC        Primer1

In [15]:
test_df.to_csv('test_df.csv', index=False)

In [7]:
from Bio.Seq import Seq

def find_sequence_location(query_seq, fasta_file):
    ref_genome = SeqIO.read(fasta_file, "fasta")
    print('done1')
    position = ref_genome.seq.find(query_seq)
    print('done2')
    
    if position != -1:
        return [position, position+len(query_seq)]
    else:
        print('!!!Primer not found in the reference genome')
        return 0

def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    print('done')
    return "".join(complement[base] for base in reverse_seq)

ref_genome = '/mnt/storage10/lwang/Projects/TOAST/toast/db/MTB-h37rv_asm19595v2-eg18.fa'

In [39]:
print(find_sequence_location('CACTCCGAAGAAGCCGAACT', ref_genome))

done1
done2
[781678, 781698]


In [46]:
full_data  = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [11]:
# primer_input = pd.read_csv('primer_input.csv')
primer_input = test_df
pLeft_ID = []
pLeft_coord = []
pLeft_length = []
pLeft_Sequences = []
pRight_ID = []
pRight_coord = []
pRight_length = []
pRight_Sequences = []
Project_size = []
Designed_ranges = []

for i, x in primer_input.iterrows():
    print(x)
    print(x[0], x[1], x[2])
    print(type(x[0]), type(x[1]), type(x[2]))
    pLeft_ID.append(x[2]+'-UserLeft')
    locl = find_sequence_location(x[0], ref_genome)[0]
    pLeft_coord.append(locl)
    pLeft_length.append(len(x[0]))
    pLeft_Sequences.append(x[0])
    pRight_ID.append(x[2]+'-UserRight')
    locr = find_sequence_location(x[1], ref_genome)[1]
    pRight_coord.append(locr)
    pRight_length.append(len(x[1]))
    pRight_Sequences.append(x[1])
    Project_size.append(locr - locl)
    Designed_ranges.append([locl, locr])
    
columns = ['pLeft_ID', 'pLeft_coord', 'pLeft_length', 'pLeft_Tm', 'pLeft_GC', 
            'pLeft_Sequences', 'pLeft_EndStability', 'pRight_ID', 'pRight_coord', 
            'pRight_length', 'pRight_Tm', 'pRight_GC', 'pRight_Sequences', 
            'pRight_EndStability', 'Penalty', 'Product_size', 'Amplicon_type', 
            'Designed_ranges']
df = pd.DataFrame(columns=columns)

df['pLeft_ID'] = pLeft_ID
df['pLeft_coord'] = pLeft_coord
df['pLeft_length'] = pLeft_length
df['pLeft_Sequences'] = pLeft_Sequences
df['pRight_ID'] = pRight_ID
df['pRight_coord'] = pRight_coord
df['pRight_length'] = pRight_length
df['pRight_Sequences'] = pRight_Sequences
df['Product_size'] = Project_size
df['Designed_ranges'] = Designed_ranges
df['pLeft_Tm'] = '-'
df['pLeft_GC'] = '-'
df['pLeft_EndStability'] = '-'
df['pRight_Tm'] = '-'
df['pRight_GC'] = '-'
df['pRight_EndStability'] = '-'
df['Penalty'] = '-'
df['Amplicon_type'] = 'User-defined'



Forward Primers    CACTCCGAAGAAGCCGAACT
Reverse Primers    TACGGATCGCAGTTGGTCAC
Name of Primer                  Primer1
Name: 0, dtype: object
CACTCCGAAGAAGCCGAACT TACGGATCGCAGTTGGTCAC Primer1
<class 'str'> <class 'str'> <class 'str'>
done1
done2
done1
done2


/tmp/ipykernel_2792191/2629762696.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(x[0], x[1], x[2])
/tmp/ipykernel_2792191/2629762696.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(type(x[0]), type(x[1]), type(x[2]))
/tmp/ipykernel_2792191/2629762696.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pLeft_ID.append(x[2]+'-UserLeft')
/tmp/ipykernel_2792191/2629762696.py:19: FutureWarning: Series.__getitem__ 

In [12]:
df

pLeft_ID  pLeft_coord  pLeft_length pLeft_Tm pLeft_GC  \
0  Primer1-UserLeft       781678            20        -        -   

        pLeft_Sequences pLeft_EndStability          pRight_ID  pRight_coord  \
0  CACTCCGAAGAAGCCGAACT                  -  Primer1-UserRight        782004   

   pRight_length pRight_Tm pRight_GC      pRight_Sequences  \
0             20         -         -  TACGGATCGCAGTTGGTCAC   

  pRight_EndStability Penalty  Product_size Amplicon_type   Designed_ranges  
0                   -       -           326  User-defined  [781678, 782004]

In [48]:
full_data.loc[full_data['genome_pos'] > locl and full_data['genome_pos'] < locl , 'weight'] = 0
full_data.loc[(full_data['genome_pos'] > locl) & (full_data['genome_pos'] < loch), 'weight'] = 0


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [13]:
def user_defined(primer_input_file: str, refgenome: str, full_data: pd.DataFrame):
  primer_input = test_df
  pLeft_ID = []
  pLeft_coord = []
  pLeft_length = []
  pLeft_Sequences = []
  pRight_ID = []
  pRight_coord = []
  pRight_length = []
  pRight_Sequences = []
  Project_size = []
  Designed_ranges = []

  for i, x in primer_input.iterrows():
    print(x)
    print(x[0], x[1], x[2])
    print(type(x[0]), type(x[1]), type(x[2]))
    pLeft_ID.append(x[2]+'-UserLeft')
    locl = find_sequence_location(x[0], ref_genome)[0]
    pLeft_coord.append(locl)
    pLeft_length.append(len(x[0]))
    pLeft_Sequences.append(x[0])
    pRight_ID.append(x[2]+'-UserRight')
    locr = find_sequence_location(reverse_complement_sequence(x[1]), ref_genome)[1]
    pRight_coord.append(locr)
    pRight_length.append(len(x[1]))
    pRight_Sequences.append(x[1])
    Project_size.append(locr - locl)
    Designed_ranges.append([locl, locr])
      
  columns = ['pLeft_ID', 'pLeft_coord', 'pLeft_length', 'pLeft_Tm', 'pLeft_GC', 
              'pLeft_Sequences', 'pLeft_EndStability', 'pRight_ID', 'pRight_coord', 
              'pRight_length', 'pRight_Tm', 'pRight_GC', 'pRight_Sequences', 
              'pRight_EndStability', 'Penalty', 'Product_size', 'Amplicon_type', 
              'Designed_ranges']
  df = pd.DataFrame(columns=columns)

  df['pLeft_ID'] = pLeft_ID
  df['pLeft_coord'] = pLeft_coord
  df['pLeft_length'] = pLeft_length
  df['pLeft_Sequences'] = pLeft_Sequences
  df['pRight_ID'] = pRight_ID
  df['pRight_coord'] = pRight_coord
  df['pRight_length'] = pRight_length
  df['pRight_Sequences'] = pRight_Sequences
  df['Product_size'] = Project_size
  df['Designed_ranges'] = Designed_ranges
  df['pLeft_Tm'] = '-'
  df['pLeft_GC'] = '-'
  df['pLeft_EndStability'] = '-'
  df['pRight_Tm'] = '-'
  df['pRight_GC'] = '-'
  df['pRight_EndStability'] = '-'
  df['Penalty'] = '-'
  df['Amplicon_type'] = 'User-defined'

  full_data.loc[(full_data['genome_pos'] > locl) & (full_data['genome_pos'] < locr), 'weight'] = 0

  primer_pool = pRight_Sequences + pLeft_Sequences
  covered_ranges = [locl, locr]

  return df, primer_pool, full_data, covered_ranges
  # Create an empty DataFrame with these columns

In [3]:
accepted_primers = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output/Primer_design-accepted_primers-2-400.csv')

In [4]:
accepted_primers

index  pLeft_ID  pLeft_coord  pLeft_length   pLeft_Tm  pLeft_GC  \
0      0  P1-sp-L1       761087            20  60.033405      50.0   
1      0  P1-ns-L1      2155074            20  59.827648      60.0   

        pLeft_Sequences  pLeft_EndStability pRight_ID  pRight_coord  \
0  AGCCAGCTGAGCCAATTCAT                2.57  P1-sp-R1        761445   
1  CTCTTCGTCAGCTCCCACTC                3.51  P1-ns-R1       2155436   

   pRight_length  pRight_Tm  pRight_GC      pRight_Sequences  \
0             18  60.726180  66.666667    CGATCGATGCGGACGGTC   
1             20  60.179081  60.000000  GGGTTCACGTAGATCAGCCC   

   pRight_EndStability   Penalty  Product_size            Amplicon_type  \
0                 3.69  2.759585           358  Gene_specific:rpoB,katG   
1                 4.11  0.351433           362             Non_specific   

      Designed_ranges  
0    [761087, 761445]  
1  [2155074, 2155436]

In [9]:
out_bed = {}
colors = ['0,0,255', '0,255,0', '255,0,0', '128,0,0']*accepted_primers.shape[0]
for i, x in accepted_primers.iterrows():
    designed_range_name = f"Designed-A{i+1}-{x['pLeft_ID'].split('-')[1]}"
    amplicone_name = f"A{i+1}-{x['pLeft_ID'].split('-')[1]}"
    # print(x['Designed_ranges'])
    designed_ranges = x['Designed_ranges'].strip('][')

    out_bed[designed_range_name] = [int(y) for y in designed_ranges.split(', ')]
    out_bed[amplicone_name] = [x['pLeft_coord'], x['pRight_coord']+x['pRight_length']]
    out_bed[x['pLeft_ID']] = [x['pLeft_coord'], x['pLeft_coord']+x['pLeft_length']]
    out_bed[x['pRight_ID']] = [x['pRight_coord'], x['pRight_coord']+x['pRight_length']]
out = pd.DataFrame(out_bed).T
out[2] = out.index
out.insert(loc = 0,
           column='col1',
           value = ['Chromosome'] * out.shape[0])

# putting in the coluns
out[4] = 0 # score 
out[5] = '.' # strand
out[6] = out[0] # thickStart
out[7] = out[1] #ThickEnd
out[8] = colors # itemRgb



out.to_csv('test.bed', sep='\t', header=False, index=False)

In [ ]:
['Chromosome'] * out.shape[0]

In [3]:
aa = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/cache/Amplicon_design_output/Primer_design-accepted_primers-2-400.csv')

In [7]:
aa.columns

Index(['index', 'pLeft_ID', 'pLeft_coord', 'pLeft_length', 'pLeft_Tm',
       'pLeft_GC', 'pLeft_Sequences', 'pLeft_EndStability', 'pRight_ID',
       'pRight_coord', 'pRight_length', 'pRight_Tm', 'pRight_GC',
       'pRight_Sequences', 'pRight_EndStability', 'Penalty', 'Product_size',
       'Amplicon_type', 'Designed_ranges'],
      dtype='object')

In [5]:
for x,y in zip(aa['pLeft_coord'].tolist(), aa['pRight_coord'].tolist()):
    print(x,y)

761087 761445
2155074 2155436


In [ ]:
out[4] = 0
out[5] = '.'
out[6] = out[1]
out[7] = out[2]


In [10]:
out

col1        0        1               2  4  5        6  \
Designed-A1-sp  Chromosome   761087   761445  Designed-A1-sp  0  .   761087   
A1-sp           Chromosome   761087   761463           A1-sp  0  .   761087   
P1-sp-L1        Chromosome   761087   761107        P1-sp-L1  0  .   761087   
P1-sp-R1        Chromosome   761445   761463        P1-sp-R1  0  .   761445   
Designed-A2-ns  Chromosome  2155074  2155436  Designed-A2-ns  0  .  2155074   
A2-ns           Chromosome  2155074  2155456           A2-ns  0  .  2155074   
P1-ns-L1        Chromosome  2155074  2155094        P1-ns-L1  0  .  2155074   
P1-ns-R1        Chromosome  2155436  2155456        P1-ns-R1  0  .  2155436   

                      7        8  
Designed-A1-sp   761445  0,0,255  
A1-sp            761463  0,255,0  
P1-sp-L1         761107  255,0,0  
P1-sp-R1         761463  128,0,0  
Designed-A2-ns  2155436  0,0,255  
A2-ns           2155456  0,255,0  
P1-ns-L1        2155094  255,0,0  
P1-ns-R1        2155456  128,0,0

In [39]:
amplicone_name

'P1-L1'

In [9]:
full_data = pd.read_csv('/mnt/storage10/lwang/Projects/TOAST/toast/db/variants.csv')

In [10]:
full_data1 = full_data[~full_data['drugs'].isna()]


In [14]:
full_data1.to_csv('variants', index=False)

In [11]:
full_data1.shape

(268368, 9)

In [6]:
pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/test/Amplicon_design_output/Primer_design-accepted_primers-42-400.csv').head().to_excel('test.xlsx')

In [7]:
genes = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/db/MTB-h37rv_asm19595v2-eg18.gff', sep = '\t', header = None, skiprows=[0,1,2,3,4,5,6])
genes = genes[genes[2]=='gene']
genes['Name'] = genes[8].str.extract(r'Name=([^;]*)')
genes = genes[[3,4,'Name']]
genes.columns = ['start', 'end', 'gene']

In [9]:
genes = genes[~genes['gene'].isna()]

In [7]:
df = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/db/variants.csv')
# df = df[~df['drugs'].isna()]
# df = df[~df['type'].isin(['synonymous_variant','non_coding_transcript_exon_variant'])]
# df = df[df['gene']=='fabG1']

# print(df['genome_pos'].min())
# print(df['genome_pos'].max())

In [10]:
df1['gene'].value_counts()/df['gene'].value_counts()

gene
PPE35      NaN
Rv1258c    NaN
Rv1979c    NaN
Rv2752c    NaN
Rv3083     NaN
Rv3236c    NaN
aftB       NaN
ahpC       NaN
ald        NaN
alr        NaN
atpE       NaN
ccsA       NaN
clpC1      NaN
ddn        NaN
eis        NaN
embA       NaN
embB       NaN
embC       NaN
embR       NaN
ethA       NaN
ethR       NaN
fabG1      1.0
fbiA       NaN
fbiB       NaN
fbiC       NaN
fbiD       NaN
fgd1       NaN
folC       NaN
fprA       NaN
gid        NaN
gyrA       NaN
gyrB       NaN
inhA       NaN
kasA       NaN
katG       NaN
mmpL5      NaN
mmpR5      NaN
mmpS5      NaN
mshA       NaN
ndh        NaN
panD       NaN
pepQ       NaN
pncA       NaN
ribD       NaN
rplC       NaN
rpoA       NaN
rpoB       NaN
rpoC       NaN
rpsA       NaN
rpsL       NaN
rrl        NaN
rrs        NaN
thyA       NaN
thyX       NaN
tlyA       NaN
ubiA       NaN
whiB6      NaN
whiB7      NaN
Name: count, dtype: float64

In [4]:
df = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/db/variants.csv')
df1 = df[df['gene']=='fabG1']
df1['change'].unique()

array(['c.-15C>T', 'c.-60C>G', 'c.-8T>C', ..., 'p.Glu59Gly', 'p.Ala43Pro',
       'p.Arg25Pro'], dtype=object)

In [17]:
import pandas as pd

# Example Series
series1 = pd.Series({'fabG1': 3, 'PPE35': 1, 'fprA': 1}, name='count')
series2 = pd.Series({'fabG1': 10, 'PPE35': 10, 'fprA': 10, 'gid': 10}, name='count')

series1_aligned = series1.reindex(series2.index).fillna(0)

# Calculate the ratio and format as string
result = series1_aligned / series2
formatted_result = result.apply(lambda x: f"{x:.0%}")

# Combine the two series
combined_series = series1_aligned.astype(str) + "/" + series2.astype(str) + " (" + formatted_result + ")"

print(combined_series)


fabG1    3.0/10 (30%)
PPE35    1.0/10 (10%)
fprA     1.0/10 (10%)
gid       0.0/10 (0%)
Name: count, dtype: object


In [11]:
df['gene'].value_counts()

gene
gyrA       670971
rrs        522322
rpoC       508252
rrl        503108
embA       289453
mmpL5      285863
rpoB       278048
clpC1      246191
embB       235966
rpsL       189191
PPE35      188896
fprA       186174
gid        172437
Rv1979c    170503
whiB6      169146
katG       165235
ald        164828
embC       161971
rpsA       161781
tlyA       149695
fgd1       135426
rpoA       129223
aftB       127931
mshA       126670
fbiC       117931
kasA       106268
gyrB        77837
Rv2752c     75833
pncA        74039
ccsA        73707
inhA        66439
Rv3236c     63067
thyA        61490
Rv3083      56273
ndh         53898
Rv1258c     50703
ethA        50536
ahpC        47889
rplC        43808
mmpS5       43471
folC        39463
thyX        37946
ubiA        36804
fbiB        35925
fabG1       33327
fbiD        32937
alr         30611
eis         27123
pepQ        25431
embR        21930
atpE        19244
whiB7       13313
panD        13007
ethR        11853
fbiA        10812
ribD 

In [12]:
df1['gene'].value_counts()

gene
fabG1    33327
Name: count, dtype: int64

In [5]:
reference_design = './test/Amplicon_design_output/Primer_design-accepted_primers-10-400.csv'
pd.read_csv(reference_design, header = 0)

pLeft_ID  pLeft_coord  pLeft_length   pLeft_Tm   pLeft_GC  pLeft_Sequences  \
0    P1-L0       761087            20  60.033405  50.000000              NaN   
1    P2-L0      2155074            20  59.827648  60.000000              NaN   
2    P3-L0      4247449            20  59.822859  55.000000              NaN   
3    P4-L0      2288903            20  60.179194  60.000000              NaN   
4    P5-L0       781654            19  60.446445  57.894737              NaN   
5    P6-L1      4247250            20  60.750911  60.000000              NaN   
6    P7-L0      1673419            20  59.970965  60.000000              NaN   
7    P8-L0         7344            20  60.109458  55.000000              NaN   
8    P9-L0      2288582            19  60.599797  63.157895              NaN   
9   P10-L0      1472411            20  60.038526  55.000000              NaN   

   pLeft_EndStability pRight_ID  pRight_coord  pRight_length  pRight_Tm  \
0                2.57     P1-R0        761445             18  60.726180   
1                3.51     P2-R0       2155436             20  60.179081   
2                2.24     P3-R0       4247838             19  60.004320   
3                3.46     P4-R0       2289254             20  60.599204   
4                2.90     P5-R0        782004             20  60.108992   
5                4.30     P6-R1       4247602             18  60.437359   
6                5.14     P7-R0       1673835             20  59.905954   
7                4.04     P8-R0          7698             20  60.179081   
8                4.45     P9-R0       2288998             20  60.108615   
9                4.35    P10-R0       1472826             20  60.037605   

   pRight_GC  pRight_Sequences  pRight_EndStability   Penalty  Product_size  \
0  66.666667               NaN                 3.69  2.759585           358   
1  60.000000               NaN                 4.11  0.351433           362   
2  52.631579               NaN                 2.85  1.181461           389   
3  60.000000               NaN                 6.13  0.778398           351   
4  55.000000               NaN                 4.02  1.555437           350   
5  66.666667               NaN                 5.19  3.188270           352   
6  50.000000               NaN                 4.06  0.123080           416   
7  60.000000               NaN                 2.92  0.288539           354   
8  60.000000               NaN                 4.34  1.708412           416   
9  55.000000               NaN                 6.01  0.076131           415   

  Amplicone_type Redesign     Designed_ranges  
0   Non_specific    ['-']    [761087, 761445]  
1   Non_specific    ['-']  [2155074, 2155436]  
2   Non_specific    ['-']  [4247449, 4247838]  
3   Non_specific    ['-']  [2288903, 2289254]  
4   Non_specific    ['-']    [781654, 782004]  
5   Non_specific    ['-']  [4247250, 4247602]  
6   Non_specific    ['-']  [1673419, 1673835]  
7   Non_specific    ['-']        [7344, 7698]  
8   Non_specific    ['-']  [2288582, 2288998]  
9   Non_specific    ['-']  [1472411, 1472826]

In [5]:
susana_design = pd.read_excel('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model2in1/amplicon_TB_2023_V2.xlsx', header = 0)
susana_design = susana_design.iloc[:-3]

In [6]:
susana_design = pd.read_excel('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model2in1/amplicon_TB_2023_V2.xlsx', header = 0)
susana_design = susana_design.iloc[:-3]
susana_design.to_csv('reference_design.csv', index = False)

In [16]:
df1 = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model2in1/test/Amplicon_design_output/Primer_design-accepted_primers-4-400.csv', header = 0)

In [17]:
def modify_primer_name(primer, amplicone_type):
    prefix = 'sp' if 'Gene_specific' in amplicone_type elif 'ns' if 'Non_specific' in amplicone_type elif 'spol' if 'spol' in amplicone_type else ''
    parts = primer.split('-')
    if len(parts) > 1:
        parts[-1] = 'L' + str(int(parts[-1][1:]) + 1)  # Increment the number
    parts.insert(1, prefix)
    return '-'.join(parts)

# Apply modifications
df1['pLeft_ID'] = df1.apply(lambda x: modify_primer_name(x['pLeft_ID'], x['Amplicone_type']), axis=1)
df1['pRight_ID'] = df1.apply(lambda x: modify_primer_name(x['pRight_ID'], x['Amplicone_type']), axis=1)

In [18]:
df1

index  pLeft_ID  pLeft_coord  pLeft_length   pLeft_Tm  pLeft_GC  \
0      0  P1-sp-L1       761087            20  60.033405      50.0   
1      0  P2-sp-L1      2155074            20  59.827648      60.0   
2      0  P1-ns-L1      4247449            20  59.822859      55.0   
3      0  P2-ns-L1      2288903            20  60.179194      60.0   

        pLeft_Sequences  pLeft_EndStability pRight_ID  pRight_coord  \
0  AGCCAGCTGAGCCAATTCAT                2.57  P1-sp-L1        761445   
1  CTCTTCGTCAGCTCCCACTC                3.51  P2-sp-L1       2155436   
2  GCCGGCTACATGTCCAACTA                2.24  P1-ns-L1       4247838   
3  CCGTTCTCGTCGACTCCTTC                3.46  P2-ns-L1       2289254   

   pRight_length  pRight_Tm  pRight_GC      pRight_Sequences  \
0             18  60.726180  66.666667    CGATCGATGCGGACGGTC   
1             20  60.179081  60.000000  GGGTTCACGTAGATCAGCCC   
2             19  60.004320  52.631579   TTACCGCCGRATTCACACT   
3             20  60.599204  60.000000  CCCGCATACGTCCACCATAC   

   pRight_EndStability   Penalty  Product_size Amplicone_type Redesign  \
0                 3.69  2.759585           358  Gene_specific    ['-']   
1                 4.11  0.351433           362  Gene_specific    ['-']   
2                 2.85  1.181461           389   Non_specific    ['-']   
3                 6.13  0.778398           351   Non_specific    ['-']   

      Designed_ranges  
0    [761087, 761445]  
1  [2155074, 2155436]  
2  [4247449, 4247838]  
3  [2288903, 2289254]

In [9]:
df1

index pLeft_ID  pLeft_coord  pLeft_length   pLeft_Tm  pLeft_GC  \
0      0    P1-L0       761087            20  60.033405      50.0   
1      0    P2-L0      2155074            20  59.827648      60.0   
2      0    P1-L0      4247449            20  59.822859      55.0   
3      0    P2-L0      2288903            20  60.179194      60.0   

        pLeft_Sequences  pLeft_EndStability pRight_ID  pRight_coord  \
0  AGCCAGCTGAGCCAATTCAT                2.57     P1-R0        761445   
1  CTCTTCGTCAGCTCCCACTC                3.51     P2-R0       2155436   
2  GCCGGCTACATGTCCAACTA                2.24     P1-R0       4247838   
3  CCGTTCTCGTCGACTCCTTC                3.46     P2-R0       2289254   

   pRight_length  pRight_Tm  pRight_GC      pRight_Sequences  \
0             18  60.726180  66.666667    CGATCGATGCGGACGGTC   
1             20  60.179081  60.000000  GGGTTCACGTAGATCAGCCC   
2             19  60.004320  52.631579   TTACCGCCGRATTCACACT   
3             20  60.599204  60.000000  CCCGCATACGTCCACCATAC   

   pRight_EndStability   Penalty  Product_size Amplicone_type Redesign  \
0                 3.69  2.759585           358  Gene_specific    ['-']   
1                 4.11  0.351433           362  Gene_specific    ['-']   
2                 2.85  1.181461           389   Non_specific    ['-']   
3                 6.13  0.778398           351   Non_specific    ['-']   

      Designed_ranges  
0    [761087, 761445]  
1  [2155074, 2155436]  
2  [4247449, 4247838]  
3  [2288903, 2289254]

In [40]:
seq = 'AGAGTTGGTGCGGCGTAA'
(dg, tm) = Thermo.simplified_tm(seq, complement_sequence(seq), TmVar)


{'Gibbs': -24.099999999999998, 'Entropy': -375.6, 'Enthalpy': -140.6, 'Gibbs5P': -5.74, 'Gibbs3P': -7.260000000000001}
<class 'dict'>
Oligo: AGAGTTGGTGCGGCGTAA
ThermoDict: {'Gibbs': -24.099999999999998, 'Entropy': -375.6, 'Enthalpy': -140.6, 'Gibbs5P': -5.74, 'Gibbs3P': -7.260000000000001}
TmVar: <class '__main__.TmVar'>
Type of ThermoDict: <class 'dict'>


NameError: name 'pd' is not defined

In [48]:
ref_genome = 'MTB-h37rv_asm19595v2-eg18.fa'

def find_sequence_location(query_seq, fasta_file=ref_genome):
    ref_genome = SeqIO.read(fasta_file, "fasta")
    position = ref_genome.seq.find(query_seq)
    if position != -1:
        return [position, position+len(query_seq)]
    else:
        print('!!!Primer not found in the reference genome')
        return 0
    
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)

def check_for_snp(seq, priority, plength, forward, cut_off=1.3):
    if forward == True:
        ploc = find_sequence_location(seq)[0]
    else:    
        ploc = find_sequence_location(reverse_complement_sequence(seq))[0]
    snp_freq = priority[(priority['genome_pos']>= ploc) & (priority['genome_pos']<=ploc+plength)]['weight']
    print(snp_freq.max())
    if snp_freq.max() > cut_off:
        return True
    else:
        return False

In [2]:
full_data = pd.read_csv('snp_priority.csv')
# check_for_snp('CTCTTCGTCAGCTCCCACTC', full_data, 20, True, 1.5)

In [3]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268368 entries, 0 to 268367
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   sample_id   268368 non-null  object 
 1   genome_pos  268368 non-null  int64  
 2   gene        268368 non-null  object 
 3   change      268368 non-null  object 
 4   freq        268368 non-null  float64
 5   type        268368 non-null  object 
 6   sublin      265459 non-null  object 
 7   drtype      268368 non-null  object 
 8   drugs       268368 non-null  object 
 9   weight      268368 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 20.5+ MB


In [1]:
full_data

NameError: name 'full_data' is not defined

In [26]:
TmVar = {
    'DivalentSaltConc': 2,
    'MonovalentSaltConc': 10,
    'dNTPConc': 0.2,
    'OligoConc': 0.5
}


In [28]:
class TmVar:
	DivalentSaltConc = 2
	MonovalentSaltConc = 10
	dNTPConc = 0.2
	OligoConc = 0.5
 

In [32]:
TmVar['111'] = 111

TypeError: 'type' object does not support item assignment

# Susana design test

In [1]:
def calculate_gc_content(sequence):
    """
    Calculate the GC content of a DNA sequence.

    Parameters:
    - sequence: The DNA sequence (string)

    Returns:
    - float: The GC content as a percentage
    """
    
    # Check if the sequence is empty
    if len(sequence) == 0:
        return 0.0
    
    # Count the number of G's and C's in the sequence
    gc_count = sequence.count('G') + sequence.count('C')
    
    # Calculate the GC content
    gc_content = (gc_count / len(sequence)) * 100
    
    return gc_content



def calculate_tm_nearest_neighbor(sequence, na_concentration):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    percent_gc = (gc_count / total_length) * 100
    tm = 81.5 + 16.6 * math.log10(na_concentration) + 0.41 * percent_gc - 675 / total_length
    return tm

def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4


In [25]:
max_gc = 0
min_gc = 100
max_dg = -100
min_dg = 100
max_tm = 0
min_tm = 100
max_tm_simple = 0
min_tm_simple = 100
for x in susana_design['forward'].tolist() + susana_design['reverse'].tolist(): 
    (dg, tm) = Thermo.simplified_tm(seq, complement_sequence(seq))
    if tm > max_tm:
        max_tm = tm
    elif tm < min_tm:
        min_tm = tm
    if dg['Gibbs'] > max_dg:
        max_dg = dg['Gibbs']
    elif dg['Gibbs'] < min_dg:
        min_dg = dg['Gibbs']
    # if calculate_tm_nearest_neighbor(x, 50) > max_tm:
    #     max_tm = calculate_tm_nearest_neighbor(x, 50)
    # elif calculate_tm_nearest_neighbor(x, 50) < min_tm:
    #     min_tm = calculate_tm_nearest_neighbor(x, 50)
    # if simplified_tm(x) > max_tm_simple:
    #     max_tm_simple = simplified_tm(x)
    # elif simplified_tm(x) < min_tm_simple:
    #     min_tm_simple = simplified_tm(x)
        
print('max_gc', max_gc)
print('min_gc', min_gc)
print('max_tm', max_tm)
print('min_tm', min_tm)
print('max_dg', max_dg)
print('min_dg', min_dg)
print('max_tm_simple', max_tm_simple)
print('min_tm_simple', min_tm_simple)

max_gc 0
min_gc 100
max_tm 63.024451980920674
min_tm 63.024451980920674
max_dg -24.099999999999998
min_dg -24.099999999999998
max_tm_simple 0
min_tm_simple 100


In [ ]:
from itertools import product

def calc_self_any(primer):
    max_score = 0
    for i in range(len(primer)):
        for j in range(i+1, len(primer)):
            score = 0
            for k, l in zip(primer[i:], primer[j:]):
                if k == l:
                    score += 1
            max_score = max(max_score, score)
    return max_score

def calc_self_end(primer):
    max_score = 0
    for i in range(len(primer)):
        score = 0
        for k, l in zip(primer[i:], reversed(primer[:len(primer) - i])):
            if k == l:
                score += 1
        max_score = max(max_score, score)
    return max_score

def calc_pair_compl_any(primer1, primer2):
    max_score = 0
    for i in range(len(primer1)):
        for j in range(len(primer2)):
            score = 0
            for k, l in zip(primer1[i:], primer2[j:]):
                if k == l:
                    score += 1
            max_score = max(max_score, score)
    return max_score

def calc_pair_compl_end(primer1, primer2):
    max_score = 0
    for i in range(len(primer1)):
        score = 0
        for k, l in zip(primer1[i:], reversed(primer2[:len(primer1) - i])):
            if k == l:
                score += 1
        max_score = max(max_score, score)
    return max_score

# Test the functions
primer1 = "ATCGATCG"
primer2 = "GCTAGCTA"

for x,y in zip(susana_design['forward'].tolist(), susana_design['reverse'].tolist()): 
    print("PRIMER_MAX_SELF_ANY for primer1:", calc_self_any(x))   
    print("PRIMER_MAX_SELF_END for x:", calc_self_end(x))   
    print("PRIMER_PAIR_MAX_COMPL_ANY:", calc_pair_compl_any(x, y))  
    print("PRIMER_PAIR_MAX_COMPL_END:", calc_pair_compl_end(x, y))  


In [13]:
ref_genome = 'MTB-h37rv_asm19595v2-eg18.fa'
def find_sequence_location(query_seq, fasta_file=ref_genome):
    ref_genome = SeqIO.read(fasta_file, "fasta")
    position = ref_genome.seq.find(query_seq)
    if position != -1:
        return [position, position+len(query_seq)]
    else:
        print('!!!Primer not found in the reference genome')
        return 0

# pLeft_coord = []
# pLeft_coord.append(find_sequence_location('CATCGCACGTCGTCTTTCCG', ref_genome)[0])

# find_sequence_location('CATCGCACGTCGTCTTTCCG')[0]
# find_sequence_location(complement_sequence('CATCGCACGTCGTCTTTCCG'))
#%%
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def check_last_n_base_pairs(seq1, seq2, n):
    """
    Check if the last n base pairs of two sequences are the same.

    Parameters:
    - seq1: The first sequence (string)
    - seq2: The second sequence (string)
    - n: The number of base pairs to compare from the end (integer)

    Returns:
    - bool: True if the last n base pairs are the same, otherwise False
    """
    
    # Check if either sequence is shorter than n
    if len(seq1) < n or len(seq2) < n:
        return False
    
    # Get the last n base pairs from each sequence
    last_n_seq1 = seq1[-n:]
    last_n_seq2 = seq2[-n:]
    
    # Compare the last n base pairs
    return last_n_seq1 == last_n_seq2

# Example usage
# result = check_last_n_base_pairs("ATCGGA", "TTTGGA", 2)
# print("Are the last 2 base pairs the same?", result)  # Output should be True

# result = check_last_n_base_pairs("ATCGGA", "TTTGTA", 2)
# print("Are the last 2 base pairs the same?", result)  # Output should be False

#%%
def has_multiple_binding_sites(sequence, genome, similarity_threshold=90, min_tm=50, max_dg=-10):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0
    n = 4
    for i in range(genome_len - seq_len + 1):
        within_tm_threshold, within_dg_threshold = False, False
        subseq = genome[i:i + seq_len]
        reverse_comp_subseq = reverse_complement_sequence(subseq)
        if calculate_similarity(subseq, sequence)>similarity_threshold:
            if check_last_n_base_pairs(subseq, sequence, n):
                # print('>>>Calculated_similarity')
                within_tm_threshold, within_dg_threshold = False, False
                tm = simplified_tm(subseq)
                dg = simplified_dg(subseq)
                
                within_tm_threshold = tm >= min_tm
                within_dg_threshold = dg <= max_dg
        elif calculate_similarity(reverse_comp_subseq, sequence)>similarity_threshold:
            if check_last_n_base_pairs(reverse_comp_subseq, sequence, n):
                # print('>>>Calculated_similarity')
                within_tm_threshold, within_dg_threshold = False, False
                tm = simplified_tm(reverse_comp_subseq)
                dg = simplified_dg(reverse_comp_subseq)
                
                within_tm_threshold = tm >= min_tm
                within_dg_threshold = dg <= max_dg
            
        if within_tm_threshold & within_dg_threshold:
            # print(genome[i:i + seq_len])
            # print('>>>within_tm_threshold & within_dg_threshold')
            count += 1
            if count > 1:
                print(max(calculate_similarity(subseq, sequence), calculate_similarity(reverse_comp_subseq, sequence)))
                return True  # Early exit if more than one binding site is found
    print(max(calculate_similarity(subseq, sequence), calculate_similarity(reverse_comp_subseq, sequence)))
    return False  # Return False if only one or no binding sites are found


In [11]:
def complement_sequence(dna_sequence):
    trans = str.maketrans('ATCG', 'TAGC')
    return dna_sequence.upper().translate(trans)

# print(complement_sequence("ATGCGTA"))
# Example usage:
# dna_sequence = "ATGCGTA"
# complement_sequence = complement_sequence(dna_sequence)
# print(complement_sequence)  # Output: TACGCAT
#%%
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)

def genome_size(fasta_file):
    total_length = 0
    with open(fasta_file, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                total_length += len(line.strip())
    return total_length

def extract_sequence_from_fasta(start_pos, end_pos, padding = 150, fasta_file= 'MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome'):
    """
    Extracts a subsequence from a FASTA file based on the given sequence ID, start position, and end position.
    """
    # Iterate over the sequences in the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Check if the current sequence ID matches the desired sequence ID
        
        if record.id == sequence_id:
            # Extract the subsequence based on the start and end positions
            subsequence = record.seq[start_pos-padding:end_pos+padding]
            return str(subsequence)  # Return the subsequence as a string
    # If the sequence ID is not found, return None
    return None
genome = extract_sequence_from_fasta(0, genome_size(ref_genome),0)



In [14]:
for x in susana_design['forward'].tolist() + susana_design['reverse'].tolist(): 
    print(has_multiple_binding_sites(x, genome, similarity_threshold=80, min_tm=50, max_dg=-10))


83.33333333333334
True
85.0
True
100.0
True
82.35294117647058
True
10.0
False
100.0
True
10.0
False
20.0
False
20.0
False
35.0
False
35.0
False
25.0
False
31.57894736842105
False
82.35294117647058
True
83.33333333333334
True
82.35294117647058
True
84.21052631578947
True
100.0
True
82.35294117647058
True
82.35294117647058
True
30.0
False
30.0
False
25.0
False
83.33333333333334
True
84.21052631578947
True
100.0
True
94.11764705882352
True
36.84210526315789
False
35.0
False
100.0
True
82.35294117647058
True
83.33333333333334
True
82.35294117647058
True
35.0
False
85.71428571428571
True
81.81818181818183
True
20.0
False
85.0
True
88.23529411764706
True
82.35294117647058
True
15.0
False
88.23529411764706
True
31.57894736842105
False
35.0
False
30.0
False
30.0
False
84.21052631578947
True
85.0
True
20.0
False
20.833333333333336
False
100.0
True
84.21052631578947
True
31.57894736842105
False
20.0
False
82.35294117647058
True
82.35294117647058
True
30.0
False
30.0
False
83.33333333333334
True


KeyboardInterrupt: 

In [ ]:
# Example usage
gc_content = calculate_gc_content("ATCGGCAT")


In [2]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO
from Bio.Blast import NCBIXML
import subprocess

In [2]:
def complement_sequence(dna_sequence):
    trans = str.maketrans('ATCG', 'TAGC')
    return dna_sequence.upper().translate(trans)

# print(complement_sequence("ATGCGTA"))
# # Example usage:
# # dna_sequence = "ATGCGTA"
# # complement_sequence = complement_sequence(dna_sequence)
# # print(complement_sequence)  # Output: TACGCAT
# #%%
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)


TACGCAT


In [ ]:
def calculate_mismatches(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def find_alignments(sequence, genome, target_site):
    alignments = []
    seq_len = len(sequence)
    genome_len = len(genome)
    
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i+seq_len]
        mismatches = calculate_mismatches(sequence, subseq)
        binding_location = (i, i+seq_len)
        distance_from_target = min(abs(i - target_site), abs(i + seq_len - target_site))
        
        alignment_info = {
            'binding_location': binding_location,
            'mismatches': mismatches,
            'distance_from_target': distance_from_target
        }
        alignments.append(alignment_info)
    
    return alignments

# Usage
sequence = 'ATG'
genome = 'ATGCGTATGACGTAG'
target_site = 7  # Assume the target site is at position 7
alignments = find_alignments(sequence, genome, target_site)

for alignment in alignments:
    binding_location = alignment['binding_location']
    mismatches = alignment['mismatches']
    distance_from_target = alignment['distance_from_target']
    print(f'Binding location: {binding_location}, Mismatches: {mismatches}, Distance from target: {distance_from_target}')



In [ ]:
def calculate_mismatches(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def find_alignments(sequence, genome, target_site):
    alignments = []
    seq_len = len(sequence)
    genome_len = len(genome)
    
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i+seq_len]
        mismatches = calculate_mismatches(sequence, subseq)
        binding_location = (i, i+seq_len)
        distance_from_target = min(abs(i - target_site), abs(i + seq_len - target_site))
        
        alignment_info = {
            'binding_location': binding_location,
            'mismatches': mismatches,
            'distance_from_target': distance_from_target
        }
        alignments.append(alignment_info)
    
    return alignments

# Usage
sequence = 'ATG'
genome = 'ATGCGTATGACGTAG'
target_site = 7  # Assume the target site is at position 7
alignments = find_alignments(sequence, genome, target_site)

for alignment in alignments:
    binding_location = alignment['binding_location']
    mismatches = alignment['mismatches']
    distance_from_target = alignment['distance_from_target']
    print(f'Binding location: {binding_location}, Mismatches: {mismatches}, Distance from target: {distance_from_target}')

In [76]:
ref_genome = '/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/MTB-h37rv_asm19595v2-eg18.fa'

def genome_size(fasta_file):
    total_length = 0
    with open(fasta_file, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                total_length += len(line.strip())
    return total_length

def extract_sequence_from_fasta(start_pos, end_pos, padding = 150, fasta_file= 'MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome'):
    """
    Extracts a subsequence from a FASTA file based on the given sequence ID, start position, and end position.
    """
    # Iterate over the sequences in the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Check if the current sequence ID matches the desired sequence ID
        
        if record.id == sequence_id:
            # Extract the subsequence based on the start and end positions
            subsequence = record.seq[start_pos-padding:end_pos+padding]
            return str(subsequence)  # Return the subsequence as a string
    # If the sequence ID is not found, return None
    return None

extract_sequence_from_fasta(0, genome_size(ref_genome))

'CGAGCCGTTGCCGGTAGGTTGCGGCTGGTTATCGACGGTACTGTCCACATTTGTGGATAGCCATGTGGACAGTTCACCTGCCCACAACAACGGTTGTAGCTCGACCCGGAACCAAGACCCGGAACTAACGAGAACCAGGGAGATACGTCG'

In [74]:
genome_size(ref_genome)

4411532

In [78]:
genome_size(ref_genome)

4411532

In [80]:
import primer_selection

TACGCAT


In [ ]:
primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)

In [83]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)  # Adds 0 if dinucleotide not found
    return dg


def is_good_primer_candidate(primer, genome, target_site, max_mismatches, binding_location_range, max_distance_from_target, min_tm, max_dg):
    primer_len = len(primer)
    genome_len = len(genome)
    
    for i in range(genome_len - primer_len + 1):
        subseq = genome[i:i+primer_len]
        mismatches = calculate_mismatches(primer, subseq)
        distance_from_target = min(abs(i - target_site), abs(i + primer_len - target_site))
        
        # Check if the alignment meets the thresholds
        within_mismatch_threshold = mismatches <= max_mismatches
        within_binding_location_range = binding_location_range[0] <= i <= binding_location_range[1]
        within_distance_threshold = distance_from_target <= max_distance_from_target
        
        # Evaluate binding strength
        tm = simplified_tm(subseq)
        dg = simplified_dg(subseq)
        strong_binding = tm >= min_tm and dg <= max_dg
        
        # If a strong alternative binding site is found, return False
        summary = within_mismatch_threshold + within_binding_location_range + within_distance_threshold + strong_binding
        # if within_mismatch_threshold and within_binding_location_range and within_distance_threshold and strong_binding:
        if summary > 3:
            return False
    return True  # No strong alternative binding sites found

# Usage
primer = 'CGAACTCGAGGCTGCCTACT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
target_site = 7
max_mismatches = 2
binding_location_range = (0, 15)
max_distance_from_target = 50
min_tm = 50  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

is_good_candidate = is_good_primer_candidate(primer, genome, target_site, max_mismatches, binding_location_range, max_distance_from_target, min_tm, max_dg)
print(is_good_candidate)


True


In [ ]:
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        tm = simplified_tm(sequence)
        dg = simplified_dg(sequence)

        within_similarity_threshold = calculate_similarity(sequence, subseq, similarity_threshold)
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

## Below is the code used from testing alternative binding

In [140]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm=60, max_dg=-10):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        if calculate_similarity(subseq, sequence)>similarity_threshold:
            tm = simplified_tm(subseq)
            dg = simplified_dg(subseq)
            
            within_tm_threshold = tm >= min_tm
            within_dg_threshold = dg <= max_dg

            if within_tm_threshold and within_dg_threshold:
                print(genome[i:i + seq_len])
                count += 1
                if count > 1:
                    return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'CGAACTCGAGGCTGCCTACT'
# primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
result = has_multiple_binding_sites(primer, genome, 81)
print(result)  # Output: True or False


CGAACTCGAGGCTGCCTACT
CGGACTCGAGGGTGCCTTCT
True


In [132]:
def has_multiple_binding_sites(sequence, genome):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        if subseq == sequence:
            print(genome[i:i + seq_len])
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'CGAACTCGAGGCTGCCTACT'
# primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
result = has_multiple_binding_sites(primer, genome)
print(result)  # Output: True or False


CGAACTCGAGGCTGCCTACT
CGAACTCGAGGCTGCCTACT
True


In [103]:
def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def has_multiple_binding_sites(sequence, genome, similarity_threshold):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        similarity = calculate_similarity(sequence, subseq)
        if similarity >= similarity_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
# primer = 'CGAACTCGAGGCTGCCTACT'
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 91  # For example, 90% similarity
result = has_multiple_binding_sites(primer, genome, similarity_threshold)
print(result)  # Output: True or False


False


In [104]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        similarity = calculate_similarity(sequence, subseq)
        tm = simplified_tm(subseq)
        dg = simplified_dg(subseq)

        # Check if the alignment meets the thresholds
        within_similarity_threshold = similarity >= similarity_threshold
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 70  # For example, 90% similarity
min_tm = 40  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

result = has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg)
print(result)  # Output: True or False


False


In [130]:
def calculate_similarity(seq1, seq2, similarity_threshold):
    required_matches = len(seq1) * similarity_threshold / 100
    matches = 0
    for a, b in zip(seq1, seq2):
        if a == b:
            matches += 1
        if matches >= required_matches:
            return True
    return False    

def simplified_tm(seq):
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def precompute_dinucleotide_params(genome):
    unique_dinucleotides = set(genome[i:i+2] for i in range(len(genome) - 1))
    tm_params = {dinuc: simplified_tm(dinuc) for dinuc in unique_dinucleotides}
    dg_params = {dinuc: simplified_dg(dinuc) for dinuc in unique_dinucleotides}
    return tm_params, dg_params

tm_params, dg_params = precompute_dinucleotide_params(genome)

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        tm = simplified_tm(sequence)
        dg = simplified_dg(sequence)

        within_similarity_threshold = calculate_similarity(sequence, subseq, similarity_threshold)
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 70  # For example, 90% similarity
min_tm = 40  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

result = not has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg)
print(result)  # Output: True or False

True


In [108]:
tm_params, dg_params = precompute_dinucleotide_params(genome)


In [118]:
tm = sum(tm_params[genome[i:i+2]] for i in range(len(primer) - 1))


In [119]:
tm

122

In [116]:
tm_params

{'CA': 6,
 'AC': 6,
 'TG': 6,
 'GG': 8,
 'TT': 4,
 'CT': 6,
 'CC': 8,
 'GA': 6,
 'AT': 4,
 'AA': 4,
 'GC': 8,
 'GT': 6,
 'AG': 6,
 'CG': 8,
 'TC': 6,
 'TA': 4}

In [4]:
def gc_content(sequence):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    gc_content_percentage = (gc_count / total_length) * 100
    return gc_content_percentage

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
gc_percentage = gc_content(sequence)
print(f'GC Content: {gc_percentage:.2f}%')


GC Content: 60.00%


In [3]:
import math

def calculate_tm_nearest_neighbor(sequence, na_concentration):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    percent_gc = (gc_count / total_length) * 100
    tm = 81.5 + 16.6 * math.log10(na_concentration) + 0.41 * percent_gc - 675 / total_length
    return tm

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
na_concentration = 0.05  # Example sodium ion concentration in mol/L
tm = calculate_tm_nearest_neighbor(sequence, na_concentration)
print(f'Melting Temperature (Tm): {tm:.2f}°C')


Melting Temperature (Tm): 50.75°C


In [5]:
def calculate_tm(sequence):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    at_count = sequence.count('A') + sequence.count('T')
    tm = (gc_count * 4) + (at_count * 2)
    return tm

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
tm = calculate_tm(sequence)
print(f'Melting Temperature (Tm): {tm}°C')


Melting Temperature (Tm): 64°C


In [2]:
spol = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/raw.githubusercontent.com_GaryNapier_spolpred_master_data_spoligotypes.csv')

In [4]:
spol.columns

Index(['id', 'pct_reads_mapped', 'num_reads_mapped', 'median_coverage',
       'main_lineage', 'sublineage', 'octal_spoligotype', 'binary_spoligotype',
       'spacer1', 'spacer2', 'spacer3', 'spacer4', 'spacer5', 'spacer6',
       'spacer7', 'spacer8', 'spacer9', 'spacer10', 'spacer11', 'spacer12',
       'spacer13', 'spacer14', 'spacer15', 'spacer16', 'spacer17', 'spacer18',
       'spacer19', 'spacer20', 'spacer21', 'spacer22', 'spacer23', 'spacer24',
       'spacer25', 'spacer26', 'spacer27', 'spacer28', 'spacer29', 'spacer30',
       'spacer31', 'spacer32', 'spacer33', 'spacer34', 'spacer35', 'spacer36',
       'spacer37', 'spacer38', 'spacer39', 'spacer40', 'spacer41', 'spacer42',
       'spacer43'],
      dtype='object')

# database check

In [3]:
# qwer = pd.read_csv('/mnt/storage7/jody/philippines/projects/all_philippines/tbprofiler_new/variants.txt')
s7_variants = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/variants.txt')

In [43]:
full_data = pd.read_csv('/mnt/storage10/jody/projects/variant_dump/variants.csv')


In [15]:
tb_drug_resistance_genes = {
    'gyrB': ['Levofloxacin'],
    'gyrA': ['Levofloxacin'],
    'mshA': ['Isoniazid'],
    'rpoB': ['Rifampicin'],
    'rpoC': ['Rifampicin'],
    'rpsL': ['Streptomycin'],
    'embR': ['Ethambutol'],
    'rrs': ['Kanamycin', 'Capreomycin', 'Amikacin', 'Streptomycin'],
    'fabG1': ['Isoniazid'],
    'inhA': ['Isoniazid'],
    'rpsA': ['Pyrazinamide'],
    'tlyA': ['Capreomycin'],
    'ndh': ['Isoniazid'],
    'katG': ['Isoniazid'],
    'pncA': ['Pyrazinamide'],
    'kasA': ['Isoniazid'],
    'eis': ['Kanamycin', 'Amikacin'],
    'ahpC': ['Isoniazid'],
    'rpoA': ['Rifampicin'],
    'panD': ['Pyrazinamide'],
    'embC': ['Ethambutol'],
    'embA': ['Ethambutol'],
    'embB': ['Ethambutol'],
    'ubiA': ['Ethambutol'],
    'gid': ['Streptomycin']
}
full_data['weight'] = full_data['freq']

full_data.loc[full_data['gene'].isin(tb_drug_resistance_genes.keys()), 'weight'] += 0.5


In [36]:
gene_names = [
    'PPE35', 'Rv1258c', 'Rv1979c', 'Rv2752c', 'Rv3083', 'Rv3236c', 'aftB', 'ahpC', 'ald', 'alr', 
    'atpE', 'ccsA', 'clpC1', 'ddn', 'eis', 'embA', 'embB', 'embC', 'embR', 'ethA', 'ethR', 'fabG1', 
    'fbiA', 'fbiB', 'fbiC', 'fbiD', 'fgd1', 'folC', 'fprA', 'gid', 'gyrA', 'gyrB', 'inhA', 'kasA', 
    'katG', 'mmpL5', 'mmpR5', 'mmpS5', 'mshA', 'ndh', 'panD', 'pepQ', 'pncA', 'ribD', 'rplC', 'rpoA', 
    'rpoB', 'rpoC', 'rpsA', 'rpsL', 'rrl', 'rrs', 'thyA', 'thyX', 'tlyA', 'ubiA', 'whiB6', 'whiB7'
]

In [ ]:
for x, n, g in zip(full_data.groupby('gene')['genome_pos'].max(), full_data.groupby('gene')['genome_pos'].min(), gene_names):
    gene_range = full_data[full_data['gene']==g]
    covered = gene_range[(gene_range['genome_pos']>=n)&(gene_range['genome_pos']<=x)]
    print('==================================')
    print(x, n)
    print(gene_range['genome_pos'].max(), gene_range['genome_pos'].min())
    print(g, len(covered), len(gene_range), len(covered)/len(gene_range))
    

In [ ]:
for x, n, g in zip(full_data.groupby('gene')['genome_pos'].max(), full_data.groupby('gene')['genome_pos'].min(), gene_names):
    gene_range = s7_variants[s7_variants['gene']==g]
    covered = gene_range[(gene_range['genome_pos']>=n)&(gene_range['genome_pos']<=x)]
    print('==================================')
    print(x, n)
    print(gene_range['genome_pos'].max(), gene_range['genome_pos'].min())
    print(g, len(covered), len(gene_range), len(covered)/len(gene_range))
    

In [21]:
overlap = set(s7_variants['sample_id'].unique().tolist()).intersection(set(full_data['sample_id'].unique().tolist()))


In [ ]:
full_data[full_data['change']=='p.Ala403Ser']

In [7]:
full_data[full_data['change']=='p.Ala403Ser']

sample_id  genome_pos  gene       change      freq  \
0         ERR9787047        6446  gyrB  p.Ala403Ser  1.000000   
141       SRR8902561        6446  gyrB  p.Ala403Ser  1.000000   
2273      ERR2815611        6446  gyrB  p.Ala403Ser  0.985075   
2336      ERR3439434        6446  gyrB  p.Ala403Ser  1.000000   
3706     SRR14782548        6446  gyrB  p.Ala403Ser  0.976190   
...              ...         ...   ...          ...       ...   
7414917   ERR2512732        6446  gyrB  p.Ala403Ser  1.000000   
7415030    ERR192047        6446  gyrB  p.Ala403Ser  1.000000   
7415606   ERR4192560        6446  gyrB  p.Ala403Ser  1.000000   
7415792   SRR1735567        6446  gyrB  p.Ala403Ser  1.000000   
7420193   ERR4188235        6446  gyrB  p.Ala403Ser  1.000000   

                     type        sublin     drtype drugs  
0        missense_variant  lineage6.1.1  Sensitive   NaN  
141      missense_variant       La1.8.1      Other   NaN  
2273     missense_variant         La1.1  Sensitive   NaN  
2336     missense_variant       La1.8.1      Other   NaN  
3706     missense_variant     La1.2.BCG      Other   NaN  
...                   ...           ...        ...   ...  
7414917  missense_variant       La1.8.1      Other   NaN  
7415030  missense_variant       La1.8.1      Other   NaN  
7415606  missense_variant  lineage6.1.1  Sensitive   NaN  
7415792  missense_variant     La1.2.BCG      Other   NaN  
7420193  missense_variant       La1.8.1      Other   NaN  

[10112 rows x 9 columns]

In [5]:
full_data.shape

(7420996, 9)

In [ ]:
full_data

In [19]:
import random

def generate_random_gene_seq(length):
    bases = ['A', 'T', 'C', 'G']
    return ''.join(random.choice(bases) for _ in range(length))

random_gene_seq = generate_random_gene_seq(300)
# print(f"Random Gene Sequence of Length 100: {random_gene_seq}")


In [23]:
random_gene_seq

'CGTCTGTACATGCCGCTCTATTACAATTATTCCACAAGGGTCAAGGGGCCGCTGCGCTCGGAACGCTCGTGATAGCGCTATCTCAACGCTATTACGGTAGATTCGACTTAAACACATACCTAAGCGCAAGGTAGCACGGCGGGCGTCTTTTAAACTTGGGCATCGCGACCACCGGCCAATTACGGTTTAGCTATCGTCTTACTTAGGCTATAGAGCGCTTTGCCAGAACAAAGGAAAGCGTAAGGATAACCACGGTTCGATGTTGATGGGCGACGACAGGTAGCTCGATCCTCCACTCCG'

In [ ]:
import primer3

# Define sequence arguments
seq_args = {
    'SEQUENCE_ID': 'example',
    'SEQUENCE_TEMPLATE': random_gene_seq,
    'SEQUENCE_INCLUDED_REGION': [15, len(random_gene_seq) - 15]
}

# Define global arguments
global_args = {
    'PRIMER_TASK': 'generic',
    'PRIMER_PICK_LEFT_PRIMER': 1,
    'PRIMER_PICK_INTERNAL_OLIGO': 1,
    'PRIMER_PICK_RIGHT_PRIMER': 1,
    'PRIMER_OPT_SIZE': 20,
    'PRIMER_MIN_SIZE': 18,
    'PRIMER_MAX_SIZE': 25,
    'PRIMER_NUM_RETURN': 5,
    'PRIMER_MIN_TM': 57.0,
    'PRIMER_MAX_TM': 63.0,
    'PRIMER_MIN_GC': 20.0,
    'PRIMER_MAX_GC': 80.0,
    'PRIMER_MAX_POLY_X': 100,
    'PRIMER_INTERNAL_MAX_POLY_X': 100,
    'PRIMER_SALT_MONOVALENT': 50.0,
    'PRIMER_DNA_CONC': 50.0,
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_MAX_SELF_ANY': 12,
    'PRIMER_MAX_SELF_END': 8,
    'PRIMER_PAIR_MAX_COMPL_ANY': 12,
    'PRIMER_PAIR_MAX_COMPL_END': 8
}

# Run the primer design
designs = primer3.design_primers(seq_args, global_args)
designs = bindings.design_primers(seq_args, global_args)

# Output results
for key, value in designs.items():
    print(f"{key}: {value}")


In [12]:
# Print to validate
print("Start of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][0])
print("Length of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][1])
print("Total length of SEQUENCE_TEMPLATE: ", len(seq_args['SEQUENCE_TEMPLATE']))

# Validate bounds
if seq_args['SEQUENCE_INCLUDED_REGION'][0] + seq_args['SEQUENCE_INCLUDED_REGION'][1] > len(seq_args['SEQUENCE_TEMPLATE']):
    print("Error: SEQUENCE_INCLUDED_REGION is out of bounds")


Start of SEQUENCE_INCLUDED_REGION:  1
Length of SEQUENCE_INCLUDED_REGION:  70
Total length of SEQUENCE_TEMPLATE:  71


In [3]:
seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCCACTGCCGATCACGACGATGTTCTTAGCGTCGTAGTCGAGGTCCTCGGGCCAGTGCTGCGGATGGATGATCGGCCCGACGAAATCCTCCGAGCCGGCGAATCTCGGCGAGTAGCCCTCGTCGTAGTTGTAGTAGCCGCTGCACAGAAAGAGGAATTCGCAGGTGAGGGCGCTGAGCGTGCCGTGGCTTTGGATGTGAACGGTCCAGCGGTTTTCCGCGGTCGACCAATCGGCACTGATCACCTTGTGGTGGAACCGGATATGCCTGTCGATTCCATACATGGCCGCGGTGCTCTTGACGTACTCGAGGATGGGCTTGCCGTCGGCGATCGCCTGCCGTCCGGTCCAGGGACGGAATCGGAAACCTAGCGTGTACATGTCGGAGTCGGAGCGAATTCCGGGATAACGGAACAAATCCCAGGTGCCGCCCATGGATTCCCGCTTTTCCAGGATGGCGTAGCTCTTGGTCGGGCAACGGTCCTGCAGGTGCCAGGCCGCGCTGACACCGGAGATTCCAGCGCCCACGATGACAACGTCGAGGTGCTCGGTCATGGATCCACGCTATCAACGTAATGTCGAGGCCGTCAACGAGATGTCGACACTATCGACACGTAGTAAGCTGCCAGGGTGACCACCTCCGCGGCCAGTCAGGCTTCGCTGCCTAGGGGCCGGCGCACCGCGCGGCCGTCCGGCGACGATCGTGAACTGGCGATCCTCGCCACCGCCGAGAACCTTCTCGAGGACCGTCCGCTGGCCGATATCTCGGTCGACGATCTGGCCAAGGGCGCCGGTATCTCGAGGCCGACGTTCTACTTCTATTTCCCATCCAAGGAAGCGGTGCTGCTGACCCTGCTGGACCGGGTGGTCAATCAAGCCGACATGGCCCTACAGACCCTTGCCGAGAATCCCGCCGACACCGACCGCGAGAACATGTGGCGCACCGGGATCAACGTGTTCTTCGAGACATTCGGGTCGCACAAGGCGGTAACCCGAGCCGGTCAGGCCGCCAGGGCAACCAGTGTCGAAGTCGCCGAACTGTGGTCGACGTTTATGCAGAAGTGGATCGCCTACACGGCCGCCGTGATCGACGCCGAACGCGACCGAGGCGCGGCGCCGCGCACCCTGCCGGCCCATGAACTGGCCACAGCGCTCAACCTGATGAACGAGCGGACGCTGTTCGCGTCATTCGCCGGCGAACAGCCCTCGGT'
len(seq)

1500

In [8]:
seq[205:230]

'CACGATGTAGGTGGGTGAGCGCTGC'

In [ ]:
ACTGCCGATCACGACGATGT

In [13]:
left_seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'
len(left_seq)

295

In [10]:
left_seq

'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'

In [12]:
primer_selection.extract_sequence_from_fasta(4327174, 4327174+20, padding=0)

'AACCGGATATGCCTGTCGAT'

In [15]:
primer_selection.extract_sequence_from_fasta(4327129, 4327129+20, padding=0)


'CGGTTTTCCGCGGTCGACCA'

In [14]:
4327129 + 295

4327424

In [36]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence):
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        if query_sequence in sequence:
            start_index = sequence.index(query_sequence)
            end_index = start_index + len(query_sequence)
            return f"Sequence found in {record.id} from position {start_index} to {end_index}"
    return "Sequence not found in the FASTA file"

# Example usage
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "ATCATCAACGGGACCGAGCG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)


Sequence found in Chromosome from position 760286 to 760306


In [40]:
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "AGTCCACCGACAAGACGCTG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)

Sequence found in Chromosome from position 760365 to 760385


In [35]:
4328229-350

4327879

In [41]:
find_sequence_location(fasta_file, 'CCGGTTCAGGCTTCACCACA')

'Sequence found in Chromosome from position 13 to 33'

In [45]:
from Bio import SeqIO

def find_sequence_location(query_sequence, position, fasta_file):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        print("Primer Sequence not found in FASTA file.")
        return 0,0

In [50]:
find_sequence_location('GCCTAAACCGGCGCTAAACC', 4326008, fasta_file='/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa')


(4325973, 4325993)

In [28]:
primer_selection.extract_sequence_from_fasta(4326924, 4326944, padding=0)


'ACTGCCGATCACGACGATGT'

In [25]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence, position):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        return "Sequence not found in the FASTA file"

In [29]:
query_sequence ='ACTGCCGATCACGACGATGT'
# query_sequence = primer_selection.reverse_complement_sequence(query_sequence)
find_sequence_location(fasta_file, query_sequence, 4327932)[0]
# print(result)

4326924

In [ ]:
4327129 4327629

In [ ]:
4327129

In [20]:
[[4326879, 4327879],
 [2282776, 2283776],
 [2285958, 2286958],
 [4246734, 4247734],
 [4244193, 4245193],
 [2280899, 2281899],
 [2152912, 2153912],
 [2284954, 2285954],
 [2747141, 2748141],
 [1834840, 1835840],
 [2150990, 2151990],
 [2277174, 2278174],
 [3067958, 3068958],
 [2147188, 2148188],
 [2138550, 2139550],
 [2281712, 2282712],
 [2283527, 2284527],
 [1416371, 1417371],
 [3840393, 3841393],
 [3073502, 3074502],
 [2149884, 2150884],
 [2276296, 2277296],
 [2279554, 2280554],
 [2287239, 2288239],
 [2141412, 2142412],
 [2140159, 2141159],
 [2148199, 2149199],
 [2144518, 2145518],
 [3086947, 3087947],
 [2128861, 2129861]]

4326629